In [1]:
!pip install -q pandas numpy sentence-transformers faiss-cpu gradio groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.

In [2]:
import os
os.environ["GROQ_API_KEY"] = "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"

In [3]:
from google.colab import files
import pandas as pd
import numpy as np
import os
import glob
import zipfile
import pickle

# Create necessary directories
!mkdir -p data/raw data/processed

# Upload datasets
print("Please upload the FairytaleQA dataset ZIP file")
uploaded = files.upload()  # Upload the FairytaleQA dataset ZIP file
zip_filename = list(uploaded.keys())[0]

# Extract the ZIP file
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall()
fairytale_dir = os.path.splitext(zip_filename)[0]  # Directory name without .zip extension
print(f"Extracted FairytaleQA dataset to {fairytale_dir}")

print("\nPlease upload the Age of Acquisition XLSX file")
uploaded = files.upload()  # Upload the Age of Acquisition dataset
aoa_filename = list(uploaded.keys())[0]
!mv "$aoa_filename" data/raw/

print("\nPlease upload the Morals CSV file")
uploaded = files.upload()  # Upload the Morals dataset
morals_filename = list(uploaded.keys())[0]
!mv "$morals_filename" data/raw/

print("\nAll datasets uploaded successfully!")

Please upload the FairytaleQA dataset ZIP file


Saving FairytaleQAData-main.zip to FairytaleQAData-main.zip
Extracted FairytaleQA dataset to FairytaleQAData-main

Please upload the Age of Acquisition XLSX file


Saving AoA_ratings_Kuperman_et_al_BRM.xlsx to AoA_ratings_Kuperman_et_al_BRM.xlsx

Please upload the Morals CSV file


Saving morals.csv to morals.csv

All datasets uploaded successfully!


In [4]:
def process_fairytale_stories(fairytale_dir, output_dir="data/processed"):
    """Process section-split stories from FairytaleQA dataset"""
    print("Processing section-split stories...")

    # Path to section-split stories - using 'all' folder
    section_path = os.path.join(fairytale_dir, "data-by-train-split", "section-stories", "all")

    # Check if directory exists
    if not os.path.exists(section_path):
        # Try alternative path structure
        section_path = glob.glob(os.path.join(fairytale_dir, "*", "data-by-train-split", "section-stories", "all"))
        if section_path:
            section_path = section_path[0]
        else:
            print(f"Could not find section-stories directory in {fairytale_dir}")
            # Try a broader search
            section_path = glob.glob(os.path.join(fairytale_dir, "**", "section-stories", "all"), recursive=True)
            if section_path:
                section_path = section_path[0]
            else:
                print("Could not find section-stories directory. Please check the dataset structure.")
                return pd.DataFrame()

    print(f"Looking for stories in {section_path}")
    story_files = glob.glob(os.path.join(section_path, "*.csv"))

    if not story_files:
        print(f"No story files found in {section_path}")
        return pd.DataFrame()

    print(f"Found {len(story_files)} story files")
    all_stories = []

    for file in story_files:
        try:
            # Read the story file
            df = pd.read_csv(file)

            # Extract story name from filename
            story_name = os.path.basename(file).replace('-story.csv', '')

            # Add story name to dataframe
            df['story_name'] = story_name

            # Add to collection
            all_stories.append(df)
        except Exception as e:
            print(f"Error processing {file}: {e}")

    if not all_stories:
        print("No valid stories processed.")
        return pd.DataFrame()

    # Combine all stories
    stories_df = pd.concat(all_stories)

    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Save processed stories
    stories_df.to_csv(os.path.join(output_dir, "fairytale_stories.csv"), index=False)
    print(f"Saved {len(stories_df)} story sections from {len(all_stories)} stories")

    return stories_df

# Process the FairytaleQA dataset
stories_df = process_fairytale_stories(fairytale_dir)
print(f"Processed stories dataframe shape: {stories_df.shape}")
print(f"Columns: {stories_df.columns.tolist()}")
print(f"Sample stories:\n{stories_df.head()}")

Processing section-split stories...
Looking for stories in FairytaleQAData-main/data-by-train-split/section-stories/all
Found 278 story files
Saved 4095 story sections from 278 stories
Processed stories dataframe shape: (4095, 3)
Columns: ['section', 'text', 'story_name']
Sample stories:
   section                                               text  \
0        1  At the time when the Tang dynasty reigned over...   
1        2  Old Dragonbeard must have been a master swords...   
2        3  When the young man reached home his thoughts w...   
3        4  Said Molo: "When she stretched out three finge...   
4        5  "Nothing easier," said Molo. "On the fifteenth...   

                           story_name  
0  how-molo-stole-the-lovely-rose-red  
1  how-molo-stole-the-lovely-rose-red  
2  how-molo-stole-the-lovely-rose-red  
3  how-molo-stole-the-lovely-rose-red  
4  how-molo-stole-the-lovely-rose-red  


In [5]:
def process_age_vocabulary(aoa_file, output_dir="data/processed"):
    """Process Age of Acquisition dataset for efficient word-age filtering"""
    print(f"Processing Age of Acquisition data from {aoa_file}")

    # Load AoA dataset
    aoa_data = pd.read_excel(aoa_file)

    # Print columns for debugging
    print(f"AoA data columns: {aoa_data.columns.tolist()}")

    # Verify "Rating.Mean" column exists (average age of acquisition)
    if 'Rating.Mean' not in aoa_data.columns:
        print("Warning: 'Rating.Mean' column not found!")
        # Try to find an alternative column
        age_columns = [col for col in aoa_data.columns if any(term in col.lower() for term in ['age', 'rating', 'mean'])]
        if age_columns:
            rating_col = age_columns[0]
            print(f"Using '{rating_col}' instead")
        else:
            print("No suitable age rating column found!")
            return None
    else:
        rating_col = 'Rating.Mean'

    # Verify "Word" column exists
    if 'Word' not in aoa_data.columns:
        print("Warning: 'Word' column not found!")
        # Try first string column
        string_cols = aoa_data.select_dtypes(include=['object']).columns
        if len(string_cols) > 0:
            word_col = string_cols[0]
            print(f"Using '{word_col}' as the word column")
        else:
            print("No suitable word column found!")
            return None
    else:
        word_col = 'Word'

    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Find the minimum and maximum ages in the dataset
    min_age = int(aoa_data[rating_col].min())
    max_age = int(aoa_data[rating_col].max())

    print(f"Age range in dataset: {min_age} to {max_age}")

    # Create age-appropriate vocabulary for all ages in the range
    age_vocab = {}

    for age in range(min_age, max_age + 1):
        # Words that are acquired at or before this age
        # (include slightly more advanced words to ensure sufficient vocabulary)
        age_words = aoa_data[aoa_data[rating_col] <= age + 1]

        # Get list of words appropriate for this age
        age_vocab[age] = age_words[word_col].dropna().str.lower().tolist()
        print(f"Age {age}: {len(age_vocab[age])} appropriate words")

    # Save as pickle for easy loading
    with open(os.path.join(output_dir, "age_vocabulary.pkl"), 'wb') as f:
        pickle.dump(age_vocab, f)

    # Create and save word-to-age mapping for more efficient lookups
    word_to_age = {}
    for age in range(min_age, max_age + 1):
        for word in age_vocab[age]:
            # Only add word if not already in dictionary
            # This ensures we store the youngest age at which a word is appropriate
            if word not in word_to_age:
                word_to_age[word] = age

    # Save word-to-age mapping
    with open(os.path.join(output_dir, "word_to_age.pkl"), 'wb') as f:
        pickle.dump(word_to_age, f)

    print(f"Age vocabulary saved to {output_dir}/age_vocabulary.pkl")
    print(f"Word-to-age mapping saved to {output_dir}/word_to_age.pkl")

    return age_vocab

# Process Age of Acquisition dataset
age_vocab = process_age_vocabulary(os.path.join("data/raw", aoa_filename))
min_age = min(age_vocab.keys()) if age_vocab else 0
print(f"Sample words for youngest age ({min_age}): {list(age_vocab[min_age])[:10]}")

Processing Age of Acquisition data from data/raw/AoA_ratings_Kuperman_et_al_BRM.xlsx
AoA data columns: ['Word', 'OccurTotal', 'OccurNum', 'Freq_pm', 'Rating.Mean', 'Rating.SD', 'Dunno']
Age range in dataset: 1 to 25
Age 1: 3 appropriate words
Age 2: 48 appropriate words
Age 3: 338 appropriate words
Age 4: 994 appropriate words
Age 5: 2043 appropriate words
Age 6: 3497 appropriate words
Age 7: 5423 appropriate words
Age 8: 8067 appropriate words
Age 9: 11267 appropriate words
Age 10: 14956 appropriate words
Age 11: 18864 appropriate words
Age 12: 22727 appropriate words
Age 13: 26004 appropriate words
Age 14: 28482 appropriate words
Age 15: 29958 appropriate words
Age 16: 30693 appropriate words
Age 17: 30995 appropriate words
Age 18: 31073 appropriate words
Age 19: 31096 appropriate words
Age 20: 31103 appropriate words
Age 21: 31103 appropriate words
Age 22: 31103 appropriate words
Age 23: 31103 appropriate words
Age 24: 31104 appropriate words
Age 25: 31104 appropriate words
Age voca

In [6]:
def process_morals_dataset(morals_file, output_dir="data/processed"):
    """Process morals dataset for story endings"""
    print(f"Processing morals from {morals_file}")

    # Load morals dataset
    morals_data = pd.read_csv(morals_file)

    # Print columns for debugging
    print(f"Morals data columns: {morals_data.columns.tolist()}")

    # Check for required columns
    lesson_col = 'Lesson' if 'Lesson' in morals_data.columns else None
    themes_col = 'Themes' if 'Themes' in morals_data.columns else None
    example_col = 'Example Scenario' if 'Example Scenario' in morals_data.columns else None

    if not lesson_col or not themes_col:
        print("Error: Required columns not found in morals dataset")
        print("Needed: 'Lesson' and 'Themes' columns")
        return pd.DataFrame()

    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Process morals data
    processed_morals = []

    for _, row in morals_data.iterrows():
        lesson = row[lesson_col]
        themes_str = row[themes_col]

        # Skip rows with missing data
        if pd.isna(lesson) or pd.isna(themes_str):
            continue

        # Extract example if available
        example = row[example_col] if example_col and not pd.isna(row[example_col]) else ""

        # Split themes (assuming comma-separated list)
        themes = [theme.strip() for theme in str(themes_str).split(',')]

        # Create an entry for each theme
        for theme in themes:
            if theme:  # Skip empty themes
                processed_morals.append({
                    'moral': lesson,
                    'theme': theme.lower(),
                    'example': example
                })

    # Create and save processed dataset
    processed_df = pd.DataFrame(processed_morals)
    processed_df.to_csv(os.path.join(output_dir, "processed_morals.csv"), index=False)

    # Print summary
    theme_counts = processed_df['theme'].value_counts()
    print(f"Processed {len(processed_df)} moral entries across {len(theme_counts)} themes")
    print("\nTop themes:")
    print(theme_counts.head(10))

    return processed_df

# Process morals dataset
morals_df = process_morals_dataset(os.path.join("data/raw", morals_filename))
print(f"Sample morals:\n{morals_df.head()}")

Processing morals from data/raw/morals.csv
Morals data columns: ['Lesson', 'Themes', 'Example Scenario']
Processed 665 moral entries across 208 themes

Top themes:
theme
respect           32
responsibility    25
kindness          24
friendship        14
understanding     14
empathy           13
consideration     13
safety            12
communication     12
learning          11
Name: count, dtype: int64
Sample morals:
                                     moral         theme  \
0               Being honest is important.       honesty   
1               Being honest is important.  truthfulness   
2               Being honest is important.         trust   
3  Treat others with kindness and respect.      kindness   
4  Treat others with kindness and respect.       respect   

                                             example  
0  A character chooses to tell the truth about a ...  
1  A character chooses to tell the truth about a ...  
2  A character chooses to tell the truth about a ... 

In [7]:
def create_faiss_indexes(stories_df, morals_df, output_dir="data/processed"):
    """Create FAISS indexes for fast vector retrieval"""
    from sentence_transformers import SentenceTransformer
    import faiss

    print("Creating FAISS indexes for fast vector retrieval...")

    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Load embedding model
    model = SentenceTransformer('all-MiniLM-L6-v2')
    print("Loaded SentenceTransformer model")

    # Create FAISS index for stories
    if not stories_df.empty and 'text' in stories_df.columns:
        print("Creating story embeddings...")
        story_texts = stories_df['text'].fillna('').tolist()
        story_embeddings = model.encode(story_texts, show_progress_bar=True)

        # Normalize embeddings
        faiss.normalize_L2(story_embeddings)

        # Create FAISS index
        story_index = faiss.IndexFlatIP(story_embeddings.shape[1])
        story_index.add(story_embeddings)

        # Save index
        faiss.write_index(story_index, os.path.join(output_dir, "stories.index"))
        print(f"Created FAISS index for {len(story_texts)} stories")
    else:
        print("No valid story data for FAISS index")

    # Create FAISS index for morals
    if not morals_df.empty and 'moral' in morals_df.columns:
        print("Creating moral embeddings...")
        moral_texts = morals_df['moral'].fillna('').tolist()
        moral_embeddings = model.encode(moral_texts, show_progress_bar=True)

        # Normalize embeddings
        faiss.normalize_L2(moral_embeddings)

        # Create FAISS index
        moral_index = faiss.IndexFlatIP(moral_embeddings.shape[1])
        moral_index.add(moral_embeddings)

        # Save index
        faiss.write_index(moral_index, os.path.join(output_dir, "morals.index"))
        print(f"Created FAISS index for {len(moral_texts)} morals")
    else:
        print("No valid morals data for FAISS index")

    print("FAISS indexes created successfully")

# Create FAISS indexes
create_faiss_indexes(stories_df, morals_df)

Creating FAISS indexes for fast vector retrieval...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

Loaded SentenceTransformer model
Creating story embeddings...


Batches:   0%|          | 0/128 [00:00<?, ?it/s]

Created FAISS index for 4095 stories
Creating moral embeddings...


Batches:   0%|          | 0/21 [00:00<?, ?it/s]

Created FAISS index for 665 morals
FAISS indexes created successfully


In [8]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import os
import pickle

class StoryRAG:
    """
    Retrieval-Augmented Generation system for dynamic children's stories.
    Uses FAISS for efficient vector search to quickly retrieve relevant content.
    """

    def __init__(self, data_dir="data/processed"):
        """Initialize with processed datasets and FAISS indexes"""
        print("Initializing StoryRAG with FAISS...")

        # Load embedding model
        self.model = SentenceTransformer('all-MiniLM-L6-v2')

        # Load stories dataset
        try:
            self.stories_df = pd.read_csv(os.path.join(data_dir, "fairytale_stories.csv"))
            print(f"Loaded {len(self.stories_df)} story sections")

            # Load FAISS index for stories
            index_path = os.path.join(data_dir, "stories.index")
            if os.path.exists(index_path):
                self.story_index = faiss.read_index(index_path)
                print(f"Loaded FAISS index with {self.story_index.ntotal} story vectors")
            else:
                self.story_index = None
                print("No story index found")
        except Exception as e:
            print(f"Error loading stories: {e}")
            self.stories_df = pd.DataFrame(columns=['section', 'text', 'story_name'])
            self.story_index = None

        # Load morals dataset
        try:
            self.morals_df = pd.read_csv(os.path.join(data_dir, "processed_morals.csv"))
            print(f"Loaded {len(self.morals_df)} morals")

            # Load FAISS index for morals
            index_path = os.path.join(data_dir, "morals.index")
            if os.path.exists(index_path):
                self.moral_index = faiss.read_index(index_path)
                print(f"Loaded FAISS index with {self.moral_index.ntotal} moral vectors")
            else:
                self.moral_index = None
                print("No morals index found")
        except Exception as e:
            print(f"Error loading morals: {e}")
            self.morals_df = pd.DataFrame(columns=['moral', 'theme'])
            self.moral_index = None

        # Load age vocabulary
        try:
            with open(os.path.join(data_dir, "age_vocabulary.pkl"), 'rb') as f:
                self.age_vocab = pickle.load(f)
            print(f"Loaded vocabulary for ages {list(self.age_vocab.keys())}")

            # Also load word-to-age mapping for faster lookups
            with open(os.path.join(data_dir, "word_to_age.pkl"), 'rb') as f:
                self.word_to_age = pickle.load(f)
            print(f"Loaded word-to-age mapping with {len(self.word_to_age)} words")
        except Exception as e:
            print(f"Error loading age vocabulary: {e}")
            self.age_vocab = {}
            self.word_to_age = {}

        # Create fallback elements for reliability
        self.default_stories = [
            "Once upon a time, a brave young girl made friends with animals in the forest.",
            "Long ago, two children found a magical object that taught them about friendship.",
            "In a small village, a child learned that helping others leads to true friendship."
        ]

        self.default_morals = [
            "True friendship means being there for others in both good and difficult times.",
            "Kindness and honesty are the foundation of lasting friendships.",
            "Sharing and cooperation make everyone happier than keeping things to yourself."
        ]

        print("StoryRAG initialization complete!")

    def get_story_by_theme(self, theme, top_k=3):
        """Retrieve stories relevant to the given theme using FAISS"""
        if self.story_index is None or self.stories_df.empty:
            # Return default stories if no index or data
            return [{"text": [s], "story_name": f"Default {i+1}"} for i, s in enumerate(self.default_stories[:top_k])]

        # Encode the theme
        query_embedding = self.model.encode([theme])

        # Normalize the query embedding
        faiss.normalize_L2(query_embedding)

        # Search the FAISS index
        distances, indices = self.story_index.search(query_embedding, top_k*3)  # Get more results to find unique stories

        # Get the unique story names from the top results
        story_names = []
        seen = set()
        for i, idx in enumerate(indices[0]):
            if idx < len(self.stories_df):
                story_name = self.stories_df.iloc[idx]['story_name']
                if story_name not in seen:
                    seen.add(story_name)
                    story_names.append((story_name, float(distances[0][i])))

        # Return the full stories (up to top_k)
        results = []
        for i, (story_name, similarity) in enumerate(story_names[:top_k]):
            story_sections = self.get_story_sections(story_name)
            if story_sections:
                results.append({
                    "story_name": story_name,
                    "sections": story_sections,
                    "similarity": similarity
                })

        # If we didn't find enough stories, add defaults
        while len(results) < top_k:
            idx = len(results)
            results.append({
                "story_name": f"Default {idx+1}",
                "sections": [self.default_stories[idx % len(self.default_stories)]],
                "similarity": 0.5
            })

        return results

    def get_story_sections(self, story_name, max_sections=None):
        """Get sections of a specific story in order"""
        if self.stories_df.empty:
            return []

        # Filter for the requested story
        story_sections = self.stories_df[self.stories_df['story_name'] == story_name]

        if story_sections.empty:
            return []

        # Sort by section number if available
        if 'section' in story_sections.columns:
            story_sections = story_sections.sort_values('section')

        # Get the text for each section
        sections = story_sections['text'].tolist()

        # Limit to max_sections if specified
        if max_sections is not None and max_sections > 0:
            sections = sections[:max_sections]

        return sections

    def get_complete_story(self, story_name):
        """Get all sections of a specific story in order"""
        return self.get_story_sections(story_name)

    def get_moral_by_theme(self, theme, top_k=3):
        """Retrieve morals relevant to the given theme using FAISS"""
        if self.moral_index is None or self.morals_df.empty:
            # Return default morals if no index or data
            return [{"moral": m, "theme": theme} for m in self.default_morals[:top_k]]

        # Try exact theme match first
        theme_matches = self.morals_df[self.morals_df['theme'].str.lower() == theme.lower()]

        if not theme_matches.empty:
            # Return exact theme matches
            results = []
            for _, row in theme_matches.iterrows():
                results.append({
                    "moral": row['moral'],
                    "theme": row['theme'],
                    "example": row.get('example', '')
                })
            return results[:top_k]

        # If no exact matches, use FAISS
        query_embedding = self.model.encode([theme])

        # Normalize the query embedding
        faiss.normalize_L2(query_embedding)

        # Search the FAISS index
        distances, indices = self.moral_index.search(query_embedding, top_k)

        # Return relevant morals
        results = []
        for i, idx in enumerate(indices[0]):
            if idx < len(self.morals_df):
                row = self.morals_df.iloc[idx]
                results.append({
                    "moral": row['moral'],
                    "theme": row['theme'],
                    "example": row.get('example', ''),
                    "similarity": float(distances[0][i])
                })

        return results

    def get_story_materials(self, theme, age, length="5min", character_name=None, character_traits=None):
        """
        Get all necessary story materials based on theme and age.
        Uses FAISS for fast retrieval of relevant content.
        """
        print(f"Getting story materials for theme: {theme}, age: {age}, length: {length}")

        # Set story parameters based on length
        if length == "5min":
            paragraphs = 4
            decision_count = 1
        else:  # 10min
            paragraphs = 8
            decision_count = 2

        # Get relevant stories using FAISS
        story_results = self.get_story_by_theme(theme)

        # Get relevant morals using FAISS
        moral_results = self.get_moral_by_theme(theme)

        # Return all materials in a structured format
        return {
            "theme": theme,
            "age": age,
            "length": length,
            "paragraphs": paragraphs,
            "decision_count": decision_count,
            "character": {
                "name": character_name,
                "traits": character_traits
            },
            "story_references": story_results,
            "morals": moral_results
        }

    def filter_age_appropriate(self, text, age):
        """
        Filter text to ensure age-appropriate vocabulary.
        If words are not in the age vocabulary, keep the original words.
        """
        if age > 10:
            age = 10  # Cap at maximum age

        if age not in self.age_vocab or not self.word_to_age:
            return text  # Return original if no vocabulary data

        # Get allowed words for this age
        allowed_words = set(self.age_vocab[age])

        # Process text word by word
        words = text.split()
        filtered = []

        for word in words:
            # Extract just the alphabetic characters for checking
            clean_word = ''.join(c.lower() for c in word if c.isalpha())

            # Keep short words, punctuation, and words in the age vocabulary
            # If not in allowed words, keep the original word instead of marking it
            filtered.append(word)

        return ' '.join(filtered)

# Initialize the StoryRAG system
print("Creating StoryRAG instance...")
rag = StoryRAG()
print("StoryRAG instance created successfully!")

Creating StoryRAG instance...
Initializing StoryRAG with FAISS...
Loaded 4095 story sections
Loaded FAISS index with 4095 story vectors
Loaded 665 morals
Loaded FAISS index with 665 moral vectors
Loaded vocabulary for ages [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
Loaded word-to-age mapping with 31104 words
StoryRAG initialization complete!
StoryRAG instance created successfully!


In [9]:
def test_story_rag():
    """Test the StoryRAG system with sample inputs"""
    print("\n=== Testing StoryRAG System ===")

    # Test with different themes
    themes = ["friendship", "courage", "adventure"]

    for theme in themes:
        print(f"\n--- Testing theme: {theme} ---")

        # Get story materials
        materials = rag.get_story_materials(theme, age=7, length="5min")

        # Print retrieved stories
        print(f"Retrieved {len(materials['story_references'])} story references")

        # Print the first two sections of each story
        for i, story in enumerate(materials['story_references']):
            print(f"\nStory {i+1}: '{story['story_name']}'")
            sections = story['sections']

            # Print first two sections or all if less than two
            for j, section in enumerate(sections[:2]):
                print(f"\nSection {j+1}:")
                print(section)

            # If there are more sections, indicate this
            if len(sections) > 2:
                print(f"\n... ({len(sections) - 2} more sections)")

        # Print retrieved morals
        print(f"\nRetrieved {len(materials['morals'])} morals")
        if materials['morals']:
            print(f"Sample moral: {materials['morals'][0]['moral']}")

    # Test age-appropriate filtering
    print("\n--- Testing age filtering ---")
    test_text = "The magnificent dragon soared through the tumultuous atmosphere, displaying extraordinary aeronautical capabilities."

    for age in [5, 7, 9]:
        filtered = rag.filter_age_appropriate(test_text, age)
        print(f"Age {age}: {filtered}")

    print("\nStoryRAG test complete!")

# Run the test
test_story_rag()


=== Testing StoryRAG System ===

--- Testing theme: friendship ---
Getting story materials for theme: friendship, age: 7, length: 5min
Retrieved 3 story references

Story 1: 'remarkable-rocket'

Section 1:
The King's son was going to be married, so there were general rejoicings.  He had waited a whole year for his bride, and at last she had arrived.  She was a Russian Princess, and had driven all the way from Finland in a sledge drawn by six reindeer.  The sledge was shaped like a great golden swan, and between the swan's wings lay the little Princess herself.  Her long ermine-cloak reached right down to her feet, on her head was a tiny cap of silver tissue, and she was as pale as the Snow Palace in which she had always lived.  So pale was she that as she drove through the streets all the people wondered.  "She is like a white rose!" they cried, and they threw down flowers on her from the balconies.

Section 2:
At the gate of the Castle the Prince was waiting to receive her.  He had d

In [10]:
!pip install groq

In [11]:
"""
Story Generation Module
----------------------
This module generates structured, age-appropriate children's stories using the Groq API
with the Mixtral model. It leverages the processed datasets:
- age_vocabulary.pkl / word_to_age.pkl: For vocabulary age-appropriateness
- fairytale_stories.csv: For inspiration and structural patterns
- stories.index: For semantic similarity lookup when needed

Dependencies:
- groq
- numpy
- typing
- pickle
- pandas
- faiss
"""

import groq
import numpy as np
import pandas as pd
import pickle
import faiss
from typing import List, Dict, Optional, Tuple, Any
import os
import re
from pathlib import Path


class StoryGenerator:
    """
    Generates age-appropriate, structured children's stories using the Groq API with LLaMA models.

    Attributes:
        groq_client: Initialized Groq client for API calls
        word_to_age: Dictionary mapping words to age of acquisition scores
        fairytale_df: DataFrame containing processed fairytale stories
        stories_index: FAISS index for story similarity search (optional)
        model_name: The Groq model to use for generation (default: llama-3.1-8b-instant)
        max_tokens: Maximum tokens to generate per API call
        temperature: Temperature for generation (controls creativity)
        data_dir: Directory containing processed data files
    """

    def __init__(
        self,
        api_key: str="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",
        data_dir: str = "data/processed",
        model_name: str = "llama-3.1-8b-instant",  # Using LLaMA 3.1 8B Instant from list
        max_tokens: int = 1024,
        temperature: float = 0.7,
        use_stories_index: bool = False
    ):
        """
        Initialize the StoryGenerator with Groq API access and configuration.

        Args:
            api_key: Groq API key
            data_dir: Directory containing processed data files (default: "data/processed")
            model_name: Groq model to use (default: llama-3.1-8b-instant)
            max_tokens: Maximum tokens for generation (default: 1024)
            temperature: Controls randomness in generation (default: 0.7)
            use_stories_index: Whether to load the FAISS stories index (default: False)
        """
        self.groq_client = groq.Client(api_key=api_key)
        self.data_dir = Path(data_dir)
        self.model_name = model_name
        self.max_tokens = max_tokens
        self.temperature = temperature

        # Load Age of Acquisition vocabulary
        try:
            with open(self.data_dir / "word_to_age.pkl", "rb") as f:
                self.word_to_age = pickle.load(f)
        except FileNotFoundError:
            try:
                with open(self.data_dir / "age_vocabulary.pkl", "rb") as f:
                    self.word_to_age = pickle.load(f)
            except FileNotFoundError:
                raise FileNotFoundError("Could not find Age of Acquisition vocabulary files.")

        # Load FairytaleQA dataset
        try:
            self.fairytale_df = pd.read_csv(self.data_dir / "fairytale_stories.csv")
        except FileNotFoundError:
            print("Warning: fairytale_stories.csv not found. Some functionality may be limited.")
            self.fairytale_df = None

        # Optionally load stories index for semantic search
        self.stories_index = None
        if use_stories_index:
            try:
                self.stories_index = faiss.read_index(str(self.data_dir / "stories.index"))
                print("Successfully loaded stories.index")
            except Exception as e:
                print(f"Warning: Could not load stories.index: {e}")
                print("Story similarity functionality will be disabled.")

    def generate_story(
        self,
        age: int,
        theme: str,
        length: str = "medium",
        characters: Optional[List[str]] = None,
        additional_context: Optional[str] = None
    ) -> Dict[str, Any]:
        """
        Generate a complete, segmented children's story based on specified parameters.

        Args:
            age: Target audience age (used for vocabulary filtering)
            theme: Main theme or topic of the story
            length: Story length - "short", "medium", or "long"
            characters: Optional list of character names to include
            additional_context: Optional additional story context or requirements

        Returns:
            Dictionary containing:
                - title: Story title
                - segments: List of story segments (3-5 sentences each)
                - metadata: Generation metadata
        """
        # Configure story length parameters
        segment_count = self._determine_segment_count(length)

        # Generate story structure
        story_structure = self._generate_story_structure(theme, age, characters, segment_count)

        # Generate story title
        title = self._generate_title(theme, characters)

        # Generate segments based on structure
        segments = self._generate_segments(story_structure, age)

        # Filter vocabulary for age appropriateness
        filtered_segments = [
            self._filter_vocabulary_by_age(segment, age)
            for segment in segments
        ]

        return {
            "title": title,
            "segments": filtered_segments,
            "metadata": {
                "age": age,
                "theme": theme,
                "length": length,
                "characters": characters or [],
                "segments_count": len(filtered_segments)
            }
        }

    def _determine_segment_count(self, length: str) -> int:
        """
        Determine number of story segments based on requested length.

        Args:
            length: "short", "medium", or "long"

        Returns:
            Number of segments to generate
        """
        length_mapping = {
            "short": (3, 5),
            "medium": (5, 8),
            "long": (8, 12)
        }

        min_segments, max_segments = length_mapping.get(length.lower(), (5, 8))
        return np.random.randint(min_segments, max_segments + 1)

    def _get_inspirational_fairytale_segments(self, theme: str, count: int = 3) -> List[str]:
        """
        Retrieve relevant fairytale segments from the FairytaleQA dataset to use as inspiration.

        Args:
            theme: Theme to find relevant segments for
            count: Number of segments to retrieve

        Returns:
            List of relevant fairytale segments
        """
        if self.fairytale_df is None:
            return []

        # Simple keyword-based search in the fairytale dataset
        # In a production system, would use embedding similarity
        theme_keywords = theme.lower().split()
        matches = []

        for _, row in self.fairytale_df.iterrows():
            # This assumes fairytale_df has 'text' column - adjust field name if different
            if 'text' in self.fairytale_df.columns:
                text = str(row['text']).lower()
                if any(keyword in text for keyword in theme_keywords):
                    matches.append(row['text'])
                    if len(matches) >= count:
                        break

        return matches[:count]

    def _generate_story_structure(
        self,
        theme: str,
        age: int,
        characters: Optional[List[str]] = None,
        segment_count: int = 6
    ) -> Dict[str, Any]:
        """
        Generate the high-level structure of the story with Mixtral, using FairytaleQA
        dataset as inspiration.

        Args:
            theme: Main story theme
            age: Target audience age
            characters: Optional list of character names
            segment_count: Number of segments to generate

        Returns:
            Dictionary with story structure elements
        """
        characters_text = ", ".join(characters) if characters else "appropriate characters"

        # Create narrative arc prompt based on segment count
        intro_segments = max(1, segment_count // 6)
        buildup_segments = max(1, segment_count // 3)
        climax_segments = max(1, segment_count // 3)
        resolution_segments = segment_count - (intro_segments + buildup_segments + climax_segments)

        # Get inspirational fairytale segments
        inspiration_segments = self._get_inspirational_fairytale_segments(theme)
        inspiration_text = ""
        if inspiration_segments:
            inspiration_text = "Here are some examples from classic fairytales for inspiration:\n"
            for i, segment in enumerate(inspiration_segments, 1):
                inspiration_text += f"Example {i}: {segment}\n\n"

        prompt = f"""
        You are an expert children's storyteller. Create a detailed story structure for a {age}-year-old child
        about the theme: "{theme}". The story will feature {characters_text}.

        {inspiration_text}

        Design a structured story outline with {segment_count} segments that follows this classic structure:
        - Introduction: {intro_segments} segments to establish setting and characters
        - Buildup/Complication: {buildup_segments} segments to develop the conflict
        - Climax: {climax_segments} segments for the most exciting part
        - Resolution: {resolution_segments} segments to provide a satisfying ending

        For each segment, provide:
        1. A brief description of what happens (2-3 sentences)
        2. Key elements to include
        3. The emotional tone

        Format your response as a structured JSON without any additional explanation:

        {{
            "introduction": [
                {{
                    "description": "Brief description of segment 1",
                    "key_elements": ["element1", "element2"],
                    "tone": "emotional tone"
                }},
                ...
            ],
            "buildup": [...],
            "climax": [...],
            "resolution": [...]
        }}
        """

        response = self.groq_client.chat.completions.create(
            model=self.model_name,
            messages=[
                {"role": "system", "content": "You are an expert children's storyteller who creates detailed, age-appropriate story structures."},
                {"role": "user", "content": prompt}
            ],
            temperature=self.temperature,
            max_tokens=self.max_tokens,
            response_format={"type": "json_object"}
        )

        # Extract and parse the JSON response
        try:
            structure_text = response.choices[0].message.content
            structure = eval(structure_text)  # Convert string to dictionary
            return structure
        except Exception as e:
            # Fallback structure if parsing fails
            print(f"Error parsing story structure: {e}")
            return {
                "introduction": [{"description": f"A story about {theme}", "key_elements": [theme], "tone": "welcoming"}],
                "buildup": [{"description": "The story develops", "key_elements": ["challenges"], "tone": "curious"}],
                "climax": [{"description": "The main event happens", "key_elements": ["excitement"], "tone": "tense"}],
                "resolution": [{"description": "Everything works out", "key_elements": ["lessons"], "tone": "satisfied"}]
            }

    def _generate_title(self, theme: str, characters: Optional[List[str]] = None) -> str:
        """
        Generate a catchy title for the story based on theme and characters.

        Args:
            theme: Main story theme
            characters: Optional list of character names

        Returns:
            Story title
        """
        character_text = f" featuring {', '.join(characters)}" if characters else ""

        prompt = f"""
        Create a short, catchy, child-friendly title for a story about {theme}{character_text}.
        The title should be memorable, appropriate for children, and capture the essence of the theme.
        Return only the title with no quotation marks or additional text.
        """

        response = self.groq_client.chat.completions.create(
            model=self.model_name,
            messages=[
                {"role": "system", "content": "You are an expert at creating engaging titles for children's stories."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.8,  # Slightly higher temperature for creative titles
            max_tokens=30  # Titles are short
        )

        title = response.choices[0].message.content.strip()
        # Remove any quotes that might be in the response
        title = title.replace('"', '').replace("'", "")
        return title

    def _generate_segments(self, story_structure: Dict[str, List[Dict]], age: int) -> List[str]:
        """
        Generate the actual story segments based on the structure, maintaining coherence
        between segments by providing context of previously generated content.

        Args:
            story_structure: The structured outline of the story
            age: Target audience age for vocabulary appropriateness

        Returns:
            List of story segments (3-5 sentences each)
        """
        segments = []

        # Combine all structure sections
        all_sections = []
        for section_name in ["introduction", "buildup", "climax", "resolution"]:
            all_sections.extend(story_structure.get(section_name, []))

        # Generate text for each segment
        for i, segment_structure in enumerate(all_sections):
            description = segment_structure.get("description", "")
            key_elements = segment_structure.get("key_elements", [])
            tone = segment_structure.get("tone", "neutral")

            # Include previously generated segments as context
            previous_context = ""
            if segments:
                # Include up to 3 previous segments to avoid overwhelming the context
                context_segments = segments[-3:] if len(segments) > 3 else segments
                previous_context = f"""
                Previously in the story:
                {' '.join(context_segments)}

                Now, continuing from there, write the next segment where:
                """
            else:
                previous_context = "Write the first segment of the story where:"

            prompt = f"""
            You're writing a story segment for {age}-year-old children.

            {previous_context}
            This segment should cover: "{description}"
            Key elements to include: {', '.join(key_elements)}
            Emotional tone: {tone}

            Write this segment in 3-5 simple, engaging sentences that a {age}-year-old would understand and enjoy.
            Use simple language, active voice, and vivid imagery.
            Focus ONLY on this segment, but ensure it flows smoothly from the previous content.
            Make sure this segment connects naturally with what came before it.
            Return only the story text with no additional commentary.
            """

            response = self.groq_client.chat.completions.create(
                model=self.model_name,
                messages=[
                    {"role": "system", "content": f"You're an expert children's author writing a story for {age}-year-olds."},
                    {"role": "user", "content": prompt}
                ],
                temperature=self.temperature,
                max_tokens=200  # Each segment is fairly short
            )

            segment_text = response.choices[0].message.content.strip()
            segments.append(segment_text)

        return segments

    def _filter_vocabulary_by_age(self, text: str, age: int) -> str:
        """
        Filter and replace advanced vocabulary words with age-appropriate alternatives
        using the Age of Acquisition dataset.

        Args:
            text: Original text to filter
            age: Target audience age

        Returns:
            Age-appropriate text with advanced words replaced
        """
        words = re.findall(r'\b\w+\b', text.lower())
        complex_words = []

        # Find complex words based on Age of Acquisition
        for word in words:
            if word in self.word_to_age:
                word_age = self.word_to_age[word]
                if word_age > age + 1:  # Allow slightly advanced vocabulary
                    complex_words.append(word)

        # If no complex words, return original text
        if not complex_words:
            return text

        # If complex words found, simplify text
        complex_words_str = ", ".join(complex_words)

        prompt = f"""
        Rewrite this text to be appropriate for a {age}-year-old child:
        "{text}"

        These words might be too advanced: {complex_words_str}
        Replace them with simpler alternatives that maintain the same meaning.
        Return only the simplified text, with no additional commentary.
        """

        response = self.groq_client.chat.completions.create(
            model=self.model_name,
            messages=[
                {"role": "system", "content": "You simplify text for young children while maintaining meaning."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3,  # Lower temperature for more reliable simplifications
            max_tokens=200
        )

        simplified_text = response.choices[0].message.content.strip()
        return simplified_text


# Example usage
if __name__ == "__main__":
    # Initialize the story generator
    generator = StoryGenerator(
        api_key=os.environ.get("GROQ_API_KEY", "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"),
        data_dir="data/processed",  # Path to your processed data files
        temperature=0.7
    )

    # Generate a story
    story = generator.generate_story(
        age=6,
        theme="friendship and sharing",
        length="short",
        characters=["Luna the rabbit", "Max the squirrel"]
    )

    # Print the generated story
    print(f"Title: {story['title']}\n")
    for i, segment in enumerate(story['segments']):
        print(f"Segment {i+1}:")
        print(segment)
        print()

Title: Luna and Maxs Big Share-a-Thon

Segment 1:
"Luna the rabbit loved playing in the warm, sunny meadow near her home. She would hop and spin, her little pink nose twitching with joy. Her best friend Max the squirrel would often join her, his bushy tail wagging excitedly as he chattered happily. Together, they'd laugh and play until the sun began to set, casting a golden glow over the sparkling stream that flowed gently through the forest."

Segment 2:
One day, while Max was busy collecting more acorns, Luna snuck into his special treasure box and grabbed a handful of his shiny acorns. She thought it would be fun to add them to her own collection, but she didn't ask Max first. When Max came back and saw that his favorite acorns were gone, his eyes grew wide with sadness and anger. "Those were my special acorns!" Max shouted, his voice trembling with upset.

Segment 3:
Max was about to tell Luna how much he missed his special acorns. But then a sneaky raccoon named Rocky snuck into t

In [12]:
"""
Branching Logic Engine
---------------------
This module tracks story evolution based on user choices, implementing a tree/graph
structure to represent the branching narrative. It handles dynamic updates as the
user progresses through the story and maintains the history of choices.

Dependencies:
- typing
- json
- uuid
"""

from typing import Dict, List, Optional, Any, Set, Tuple
import json
import uuid
import os
from pathlib import Path
from datetime import datetime


class StoryNode:
    """
    Represents a single node in the story graph, containing a segment of the story
    and potential decision points.

    Attributes:
        node_id: Unique identifier for the node
        segment: Text content of this story segment
        decisions: Decision point information if this node has choices
        parent_id: ID of the parent node
        children: Dictionary mapping choice indices to child node IDs
        metadata: Additional information about this node
    """

    def __init__(
        self,
        segment: str,
        node_id: Optional[str] = None,
        parent_id: Optional[str] = None,
        decisions: Optional[Dict[str, Any]] = None,
        metadata: Optional[Dict[str, Any]] = None
    ):
        """
        Initialize a story node.

        Args:
            segment: Text content of this story segment
            node_id: Unique identifier (generated if not provided)
            parent_id: ID of the parent node (None for root)
            decisions: Decision point information if this node has choices
            metadata: Additional information about this node
        """
        self.node_id = node_id if node_id else str(uuid.uuid4())
        self.segment = segment
        self.decisions = decisions
        self.parent_id = parent_id
        self.children = {}  # Maps choice indices to child node IDs
        self.metadata = metadata if metadata else {}

    def to_dict(self) -> Dict[str, Any]:
        """
        Convert the node to a dictionary representation.

        Returns:
            Dictionary representation of the node
        """
        return {
            "node_id": self.node_id,
            "segment": self.segment,
            "decisions": self.decisions,
            "parent_id": self.parent_id,
            "children": self.children,
            "metadata": self.metadata
        }

    @classmethod
    def from_dict(cls, data: Dict[str, Any]) -> 'StoryNode':
        """
        Create a StoryNode from a dictionary representation.

        Args:
            data: Dictionary containing node data

        Returns:
            StoryNode instance
        """
        node = cls(
            segment=data["segment"],
            node_id=data["node_id"],
            parent_id=data["parent_id"],
            decisions=data.get("decisions"),
            metadata=data.get("metadata", {})
        )
        node.children = data.get("children", {})
        return node

    def add_child(self, choice_index: int, child_id: str) -> None:
        """
        Add a child node for a specific choice.

        Args:
            choice_index: Index of the choice that leads to this child
            child_id: ID of the child node
        """
        self.children[str(choice_index)] = child_id

    def has_decisions(self) -> bool:
        """
        Check if this node has decisions/choices.

        Returns:
            Boolean indicating if node has decisions
        """
        return self.decisions is not None and len(self.decisions) > 0


class BranchingLogicEngine:
    """
    Manages the branching narrative structure, tracking story evolution based on user choices.

    Attributes:
        nodes: Dictionary mapping node IDs to StoryNode objects
        current_node_id: ID of the current active node
        root_node_id: ID of the root/starting node
        story_metadata: Metadata about the overall story
        path_history: Ordered list of node IDs in the current path
        choice_history: List of choices made by the user
    """

    def __init__(
        self,
        initial_segment: Optional[str] = None,
        story_metadata: Optional[Dict[str, Any]] = None,
        api_key: Optional[str] = "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" # api_key for summarization
    ):
        """
        Initialize the branching logic engine.

        Args:
            initial_segment: Text of the first story segment (optional)
            story_metadata: Metadata about the overall story (optional)
        """
        self.nodes = {}
        self.current_node_id = None
        self.root_node_id = None
        self.story_metadata = story_metadata if story_metadata else {}
        self.path_history = []
        self.choice_history = []
        self.api_key = api_key or os.environ.get("GROQ_API_KEY")
        self.groq_client = groq.Client(api_key=self.api_key) if self.api_key else None
        self.summarization_model = "llama-3.1-8b-instant" # Or choose another model
        self.max_context_length_words = 400 # Example threshold before summarizing
        self.summary_cache = {} # Cache summaries to avoid repeated API calls

        # Create initial node if provided
        if initial_segment:
            self.initialize_story(initial_segment)

    def initialize_story(self, initial_segment: str, metadata: Optional[Dict[str, Any]] = None) -> str:
        """
        Initialize the story with the first segment.

        Args:
            initial_segment: Text of the first story segment
            metadata: Metadata for the root node

        Returns:
            ID of the created root node
        """
        # Create root node
        root_node = StoryNode(
            segment=initial_segment,
            metadata=metadata
        )

        # Store node
        self.nodes[root_node.node_id] = root_node
        self.root_node_id = root_node.node_id
        self.current_node_id = root_node.node_id
        self.path_history = [root_node.node_id]

        return root_node.node_id

    def add_decision_point(
        self,
        node_id: str,
        decision_data: Dict[str, Any]
    ) -> None:
        """
        Add a decision point to an existing node.

        Args:
            node_id: ID of the node to add decisions to
            decision_data: Decision point data (prompt and options)
        """
        if node_id not in self.nodes:
            raise ValueError(f"Node with ID {node_id} not found")

        self.nodes[node_id].decisions = decision_data

    def add_decision_to_current_node(self, decision_data: Dict[str, Any]) -> None:
        """
        Add a decision point to the current node.

        Args:
            decision_data: Decision point data (prompt and options)
        """
        if not self.current_node_id:
            raise ValueError("No current node exists")

        self.add_decision_point(self.current_node_id, decision_data)

    def add_story_continuation(
        self,
        choice_index: int,
        continuation_text: str,
        metadata: Optional[Dict[str, Any]] = None,
        log_choice: bool = True
    ) -> str:
        """
        Add a continuation to the story based on a selected choice.

        Args:
            choice_index: Index of the selected choice (0 or 1)
            continuation_text: Text for the next story segment
            metadata: Metadata for the new node

        Returns:
            ID of the newly created node
        """
        if not self.current_node_id:
            raise ValueError("No current node exists")

        current_node = self.nodes[self.current_node_id]

        # Create new node for continuation
        new_node = StoryNode(
            segment=continuation_text,
            parent_id=self.current_node_id,
            metadata=metadata
        )

        # Store the new node
        self.nodes[new_node.node_id] = new_node

        # Link current node to new node through the choice
        current_node.add_child(choice_index, new_node.node_id)

        # Update current node and path history
        self.current_node_id = new_node.node_id
        self.path_history.append(new_node.node_id)
        if log_choice:
          self.choice_history.append({
              "from_node": current_node.node_id,
              "choice_index": choice_index,
              "to_node": new_node.node_id
          })

        return new_node.node_id

    def get_current_node(self) -> StoryNode:
        """
        Get the current active node.

        Returns:
            Current StoryNode
        """
        if not self.current_node_id or self.current_node_id not in self.nodes:
            raise ValueError("Current node is not valid")

        return self.nodes[self.current_node_id]

    def get_current_segment(self) -> str:
        """
        Get the text of the current story segment.

        Returns:
            Text of the current segment
        """
        return self.get_current_node().segment

    def get_current_decisions(self) -> Optional[Dict[str, Any]]:
        """
        Get the decision point at the current node, if any.

        Returns:
            Decision data or None if no decisions exist
        """
        return self.get_current_node().decisions

    def get_path_to_current_node(self) -> List[StoryNode]:
        """
        Get the list of nodes from root to current node.

        Returns:
            List of StoryNode objects in path order
        """
        return [self.nodes[node_id] for node_id in self.path_history]

    def get_full_story_text(self) -> str:
        """
        Get the full text of the story along the current path.

        Returns:
            Combined text of all segments in the current path
        """
        path_nodes = self.get_path_to_current_node()
        return "\n\n".join([node.segment for node in path_nodes])
    def get_story_context_for_llm(self) -> str:
        """
        Gets the story context, summarizing if it exceeds the threshold.
        """
        full_text = self.get_full_story_text()
        word_count = len(full_text.split())

        # Use cache key based on the current node ID (representing story state)
        cache_key = self.current_node_id

        if word_count <= self.max_context_length_words:
            return full_text # Return full text if short enough
        elif cache_key in self.summary_cache:
             return self.summary_cache[cache_key] # Return cached summary
        elif not self.groq_client:
             print("Warning: No Groq client available for summarization. Returning full text.")
             return full_text # Fallback if no client
        else:
            # Generate summary
            print(f"Context length ({word_count} words) exceeds threshold ({self.max_context_length_words}). Summarizing...")
            prompt = f"""
            Summarize the following children's story progress concisely, capturing the key plot points, character actions, and the current situation. Keep the summary under 300 words.

            STORY SO FAR:
            {full_text}

            CONCISE SUMMARY:
            """
            try:
                response = self.groq_client.chat.completions.create(
                    model=self.summarization_model,
                    messages=[
                        {"role": "system", "content": "You are an expert at summarizing children's stories."},
                        {"role": "user", "content": prompt}
                    ],
                    temperature=0.5,
                    max_tokens=400
                )
                summary = response.choices[0].message.content.strip()
                self.summary_cache[cache_key] = summary # Cache the new summary
                return summary
            except Exception as e:
                print(f"Error during summarization: {e}. Returning full text.")
                return full_text

    def can_make_choice(self) -> bool:
        """
        Check if the current node has choices available.

        Returns:
            Boolean indicating if choices can be made
        """
        return self.get_current_node().has_decisions()

    def reset_to_node(self, node_id: str) -> None:
        """
        Reset the current position to a specific node.

        Args:
            node_id: ID of the node to reset to
        """
        if node_id not in self.nodes:
            raise ValueError(f"Node with ID {node_id} not found")

        # Find the path from root to this node
        new_path = self._find_path_to_node(node_id)
        if not new_path:
            raise ValueError(f"No valid path found to node {node_id}")

        self.current_node_id = node_id
        self.path_history = new_path

        # Update choice history to match the new path
        self._update_choice_history()

    def _find_path_to_node(self, target_node_id: str) -> Optional[List[str]]:
        """
        Find a path from the root node to the target node.

        Args:
            target_node_id: ID of the target node

        Returns:
            List of node IDs forming the path, or None if no path exists
        """
        if target_node_id == self.root_node_id:
            return [self.root_node_id]

        # Keep track of visited nodes during BFS
        visited = set()
        queue = [(self.root_node_id, [self.root_node_id])]

        while queue:
            node_id, path = queue.pop(0)

            if node_id == target_node_id:
                return path

            if node_id in visited:
                continue

            visited.add(node_id)

            # Check all children
            node = self.nodes[node_id]
            for child_id in node.children.values():
                if child_id not in visited:
                    queue.append((child_id, path + [child_id]))

        return None

    def _update_choice_history(self) -> None:
        """
        Update the choice history to match the current path.
        """
        self.choice_history = []

        for i in range(1, len(self.path_history)):
            parent_id = self.path_history[i-1]
            child_id = self.path_history[i]
            parent_node = self.nodes[parent_id]

            # Find which choice leads to this child
            choice_index = None
            for idx, cid in parent_node.children.items():
                if cid == child_id:
                    choice_index = int(idx)
                    break

            if choice_index is not None:
                self.choice_history.append({
                    "from_node": parent_id,
                    "choice_index": choice_index,
                    "to_node": child_id
                })

    def export_to_dict(self) -> Dict[str, Any]:
        """
        Export the entire story structure to a dictionary.

        Returns:
            Dictionary representation of the story structure
        """
        nodes_dict = {node_id: node.to_dict() for node_id, node in self.nodes.items()}

        return {
            "nodes": nodes_dict,
            "root_node_id": self.root_node_id,
            "current_node_id": self.current_node_id,
            "path_history": self.path_history,
            "choice_history": self.choice_history,
            "story_metadata": self.story_metadata
        }

    def export_to_json(self, pretty: bool = True) -> str:
        """
        Export the entire story structure to a JSON string.

        Args:
            pretty: Whether to format the JSON for readability

        Returns:
            JSON string representation of the story structure
        """
        data = self.export_to_dict()

        if pretty:
            return json.dumps(data, indent=2)
        else:
            return json.dumps(data)

    def save_to_file(self, filepath: str) -> None:
        """
        Save the story structure to a JSON file.

        Args:
            filepath: Path to save the file
        """
        data = self.export_to_dict()

        with open(filepath, 'w') as f:
            json.dump(data, f, indent=2)

    @classmethod
    def load_from_dict(cls, data: Dict[str, Any]) -> 'BranchingLogicEngine':
        """
        Create a BranchingLogicEngine from a dictionary representation.

        Args:
            data: Dictionary containing engine data

        Returns:
            BranchingLogicEngine instance
        """
        engine = cls(api_key=data.get("story_metadata", {}).get("api_key"))

        # Load nodes
        for node_id, node_data in data.get("nodes", {}).items():
            engine.nodes[node_id] = StoryNode.from_dict(node_data)

        # Load other attributes
        engine.root_node_id = data.get("root_node_id")
        engine.current_node_id = data.get("current_node_id")
        engine.path_history = data.get("path_history", [])
        engine.choice_history = data.get("choice_history", [])
        engine.story_metadata = data.get("story_metadata", {})

        return engine

    @classmethod
    def load_from_json(cls, json_str: str) -> 'BranchingLogicEngine':
        """
        Create a BranchingLogicEngine from a JSON string.

        Args:
            json_str: JSON string containing engine data

        Returns:
            BranchingLogicEngine instance
        """
        data = json.loads(json_str)
        return cls.load_from_dict(data)

    @classmethod
    def load_from_file(cls, filepath: str) -> 'BranchingLogicEngine':
        """
        Create a BranchingLogicEngine from a JSON file.

        Args:
            filepath: Path to the JSON file

        Returns:
            BranchingLogicEngine instance
        """
        with open(filepath, 'r') as f:
            data = json.load(f)

        return cls.load_from_dict(data)


# Example usage
if __name__ == "__main__":
    # Create a new branching story
    engine = BranchingLogicEngine(
        initial_segment="Once upon a time, there was a brave little rabbit named Luna who lived at the edge of the Whispering Forest.",
        story_metadata={"title": "Luna's Forest Adventure", "protagonist": "Luna", "theme": "bravery"}
    )

    # Add a decision point to the first segment
    engine.add_decision_to_current_node({
        "decision_prompt": "What should Luna do on this bright morning?",
        "options": [
            {
                "choice_text": "Explore deeper into the forest",
                "consequence_summary": "Luna might discover something amazing but could face dangers."
            },
            {
                "choice_text": "Visit her friend in the meadow",
                "consequence_summary": "Luna will have a safer day but might miss an adventure."
            }
        ]
    })

    # User chooses option 0 (explore the forest)
    engine.add_story_continuation(
        choice_index=0,
        continuation_text="Luna decided to be brave and explore deeper into the Whispering Forest. She hopped past tall trees and colorful flowers she had never seen before. The forest seemed to come alive around her with strange sounds and movements."
    )

    # Add another decision
    engine.add_decision_to_current_node({
        "decision_prompt": "Luna hears a strange noise coming from behind a large bush. What should she do?",
        "options": [
            {
                "choice_text": "Investigate the noise",
                "consequence_summary": "Luna might discover something interesting or dangerous."
            },
            {
                "choice_text": "Take a different path",
                "consequence_summary": "Luna will avoid potential danger but miss a discovery."
            }
        ]
    })

    # User chooses option 1 (take different path)
    engine.add_story_continuation(
        choice_index=1,
        continuation_text="Luna decided it was safer to take a different path. She turned away from the mysterious bush and followed a winding dirt trail instead. The trail led her to a small bubbling stream with stepping stones."
    )

    # Print the full story so far
    print("Full story so far:")
    print(engine.get_full_story_text())

    # Export to JSON
    story_json = engine.export_to_json()
    print("\nJSON representation:")
    print(story_json)

    # Example of saving and loading
    temp_file = "temp_story.json"
    engine.save_to_file(temp_file)

    # Load the story back
    loaded_engine = BranchingLogicEngine.load_from_file(temp_file)

    # Verify it loaded correctly
    print("\nVerifying loaded story:")
    print(f"Root node ID matches: {loaded_engine.root_node_id == engine.root_node_id}")
    print(f"Current node ID matches: {loaded_engine.current_node_id == engine.current_node_id}")
    print(f"Number of nodes matches: {len(loaded_engine.nodes) == len(engine.nodes)}")

    # Clean up temp file
    if os.path.exists(temp_file):
        os.remove(temp_file)

Full story so far:
Once upon a time, there was a brave little rabbit named Luna who lived at the edge of the Whispering Forest.

Luna decided to be brave and explore deeper into the Whispering Forest. She hopped past tall trees and colorful flowers she had never seen before. The forest seemed to come alive around her with strange sounds and movements.

Luna decided it was safer to take a different path. She turned away from the mysterious bush and followed a winding dirt trail instead. The trail led her to a small bubbling stream with stepping stones.

JSON representation:
{
  "nodes": {
    "a8fb11bf-b4a5-4c04-85ed-d07c961440f0": {
      "node_id": "a8fb11bf-b4a5-4c04-85ed-d07c961440f0",
      "segment": "Once upon a time, there was a brave little rabbit named Luna who lived at the edge of the Whispering Forest.",
      "decisions": {
        "decision_prompt": "What should Luna do on this bright morning?",
        "options": [
          {
            "choice_text": "Explore deeper in

In [13]:
"""
Decision Point Generation Module (Prompt-Based)
----------------------------------------------
This module creates meaningful decision points after each story segment,
generating two distinct branching options that influence story direction.
It uses LLaMA 3 via Groq API with a pure prompting approach that doesn't
depend on external datasets.

Dependencies:
- groq
- typing
"""

import groq
from typing import List, Dict, Optional, Any
import os
import random


import groq
from typing import List, Dict, Optional, Any, Set, Tuple
import os
import random

# Make sure BranchingLogicEngine is defined or imported before this class
# from your_branching_logic_module import BranchingLogicEngine # Assuming it's in another file/cell

class DecisionPointGenerator:
    """
    Generates meaningful, coherent decision points for interactive storytelling using LLaMA 3
    with a template-based prompting approach. Incorporates moral guidance and recovery paths.

    Attributes:
        groq_client: Initialized Groq client for API calls
        model_name: The Groq model to use for generation (default: llama3-8b-8192)
        temperature: Temperature for generation (controls creativity)
        decision_templates: Pre-defined templates for different decision types
        narrative_arc_templates: Templates for different stages of the narrative arc including moral flags
        needs_recovery: Flag to indicate if the story needs a recovery path after a wrong moral choice
    """

    def __init__(
        self,
        api_key: str,
        model_name: str = "llama3-8b-8192", # Changed default to llama3
        temperature: float = 0.75
    ):
        """
        Initialize the DecisionPointGenerator with Groq API access and configuration.

        Args:
            api_key: Groq API key
            model_name: Groq model to use (default: llama3-8b-8192)
            temperature: Controls randomness in generation (default: 0.75)
        """
        self.groq_client = groq.Client(api_key=api_key)
        self.model_name = model_name
        self.temperature = temperature

        # Templates for different types of decisions (Unchanged from original)
        self.decision_templates = {
            "action": [
                "What should {character} do next?",
                "How should {character} respond to this situation?",
                "Which path should {character} choose?",
                "What action should {character} take?",
                "Should {character} go left or right?"
            ],
            "social": [
                "Who should {character} ask for help?",
                "How should {character} approach the other character?",
                "Should {character} share what they know with others?",
                "Should {character} trust the stranger?",
                "How should {character} make new friends?"
            ],
            "problem_solving": [
                "How should {character} solve this problem?",
                "What tool should {character} use?",
                "Should {character} try to fix it alone or seek help?",
                "What approach should {character} take to overcome this obstacle?",
                "How can {character} find what they're looking for?"
            ],
            "emotional": [
                "Should {character} be brave or cautious?",
                "Should {character} follow their heart or their head?",
                "Should {character} express their feelings or keep them hidden?",
                "Should {character} forgive or hold a grudge?",
                "How should {character} deal with their fear?"
            ],
            "moral": [
                "Should {character} tell the truth or keep a secret?",
                "Should {character} follow the rules or break them for a good reason?",
                "Should {character} put others first or take care of themselves?",
                "Should {character} keep a promise even if it's difficult?",
                "Is it better for {character} to be kind or to be right?"
            ]
        }

        # Templates for different stages of the narrative arc (Merged)
        self.narrative_arc_templates = {
             "introduction": {
                 "decision_types": ["action", "social"],
                 "importance": "establishing character traits and motivations",
                 "consequence_scope": "affect how the character approaches future events",
                 "moral_guidance_required": False # Added
             },
             "rising_action": {
                 "decision_types": ["action", "problem_solving", "social"],
                 "importance": "developing the central conflict",
                 "consequence_scope": "complicate the story and raise the stakes",
                 "moral_guidance_required": False # Added
             },
             "climax": {
                 "decision_types": ["problem_solving", "emotional", "moral"],
                 "importance": "determining how the main conflict will be resolved",
                 "consequence_scope": "dramatically affect the story outcome",
                 "moral_guidance_required": True, # Added
                 "moral_choice_index": 0,  # Added: Index of the morally correct choice
                 "recovery_required": True  # Added: Requires a recovery path if wrong choice selected
             },
             "resolution": {
                 "decision_types": ["moral", "emotional", "social"],
                 "importance": "reflecting the story's theme or moral",
                 "consequence_scope": "conclude character arcs and reinforce the story's message",
                 "moral_guidance_required": True, # Added
                 "moral_choice_index": 0,  # Added: Index of the morally correct choice
                 "recovery_required": False # Added: No recovery needed in resolution
             }
         }

        # Track if we need a recovery path (Added)
        self.needs_recovery = False

    # Added method from new code
    def get_decision_point_count(self, length: str) -> int:
        """
        Determine the minimum required decision points based on story length.

        Args:
            length: "short", "medium", or "long"

        Returns:
            Minimum number of decision points required
        """
        if length.lower() == "short":
            return 2  # Minimum 2 decisions for short stories
        elif length.lower() == "medium":
            return 3  # Minimum 3 decisions for medium stories (Adjusted from 3-4)
        else:  # "long"
            return 5  # Minimum 5 decisions for long stories

    # Unchanged method from original
    def _get_narrative_stage(self, segment_index: int, total_segments: int) -> str:
        """
        Determine the current narrative stage based on segment index.

        Args:
            segment_index: Current segment index
            total_segments: Total number of segments in the story

        Returns:
            String indicating the narrative stage
        """
        progress = segment_index / total_segments if total_segments > 0 else 0.5

        if progress < 0.25:
            return "introduction"
        elif progress < 0.6:
            return "rising_action"
        elif progress < 0.85:
            return "climax"
        else:
            return "resolution"

    # Merged method
    def generate_decision_points(
        self,
        story_segment: str,
        branching_engine: 'BranchingLogicEngine', # Use BranchingLogicEngine instance
        character_names: List[str],
        age: int,
        segment_index: int = 0,
        total_segments: int = 1,
        story_length: str = "medium", # Keep story_length parameter
        is_recovery_point: bool = False # Added parameter
    ) -> Dict[str, Any]:
        """
        Generate two coherent branching options based on the current story segment.

        Args:
            story_segment: The current segment of the story
            branching_engine: Instance of the BranchingLogicEngine to get context
            character_names: List of character names in the story
            age: Target audience age for appropriate decisions
            segment_index: Current segment index (used for narrative progression)
            total_segments: Total number of segments in the story
            story_length: Length of the story ("short", "medium", or "long")
            is_recovery_point: Whether this is a recovery point after a moral choice

        Returns:
            Dictionary containing decision prompt, options, and metadata
        """
        # Get potentially summarized context from the engine
        story_context_for_llm = branching_engine.get_story_context_for_llm()

        # Determine narrative stage
        narrative_stage = self._get_narrative_stage(segment_index, total_segments)
        stage_info = self.narrative_arc_templates[narrative_stage]

        # Extract main character from list or use generic
        main_character = character_names[0] if character_names else "the main character"

        # Select appropriate decision types for this narrative stage
        decision_types = stage_info["decision_types"]
        selected_type = random.choice(decision_types)

        # Get templates for selected decision type
        templates = self.decision_templates[selected_type]

        # Select a random template and format with character name
        template_examples = "\n".join([t.format(character=main_character) for t in templates[:3]])

        # Check if this is a moral decision point (Added logic)
        is_moral_decision = stage_info.get("moral_guidance_required", False)
        moral_choice_index = stage_info.get("moral_choice_index", 0) if is_moral_decision else 0
        needs_recovery_flag = stage_info.get("recovery_required", False) if is_moral_decision else False

        # For recovery points, adjust the decision to guide back (Added logic)
        if is_recovery_point:
             selected_type = "moral"
             templates = self.decision_templates["moral"]
             template_examples = "\n".join([t.format(character=main_character) for t in templates[:3]])
             is_moral_decision = True
             moral_choice_index = 0
             needs_recovery_flag = False # Recovery point itself doesn't require another recovery

        # Customize age-appropriate language (Unchanged from original)
        if age <= 5:
            complexity = "very simple with clear choices that a 4-5 year old would understand"
        elif age <= 8:
            complexity = "simple but slightly more nuanced that a 6-8 year old would understand"
        else:
            complexity = "moderately complex with some nuance that a 9-12 year old would understand"

        # Additional context for moral decisions (Added logic)
        moral_guidance = ""
        if is_moral_decision:
            moral_guidance = f"""
            IMPORTANT: This is a moral decision point that teaches the child a lesson.

            Option {moral_choice_index + 1} should represent the morally correct choice that teaches the intended lesson.
            Option {2 - moral_choice_index} should represent an incorrect choice that goes against the moral.

            The consequence_summary for the correct choice should show a positive outcome.
            The consequence_summary for the incorrect choice should show why it's not the best decision,
            but avoid being too negative or scary.
            """

        # Use story_context_for_llm from engine instead of passed context
        prompt = f"""
        Create a decision point for a children's interactive story that is {complexity}.

        STORY CONTEXT:
        {story_context_for_llm}

        CURRENT SEGMENT:
        {story_segment}

        We are at the {narrative_stage.replace('_', ' ')} stage of the story.
        The decisions here should focus on {stage_info["importance"]}.
        The consequences should {stage_info["consequence_scope"]}.

        Here are some example decision patterns you could adapt:
        {template_examples}

        The decision should involve the character "{main_character}" and present TWO clearly different paths.
        {moral_guidance}

        FORMAT YOUR RESPONSE AS JSON:
        {{
            "decision_prompt": "An engaging question like 'What should {main_character} do?'",
            "options": [
                {{
                    "choice_text": "First option (5-10 words)",
                    "consequence_summary": "Brief hint about outcome (1 sentence)"
                }},
                {{
                    "choice_text": "Second contrasting option (5-10 words)",
                    "consequence_summary": "Different outcome hint (1 sentence)"
                }}
            ]
        }}

        IMPORTANT:
        1. Make sure both options are age-appropriate for a {age}-year-old
        2. The options should be genuinely different paths, not slight variations
        3. Each option should lead to meaningfully different story directions
        4. Use language a {age}-year-old would understand
        """

        response = self.groq_client.chat.completions.create(
            model=self.model_name,
            messages=[
                {"role": "system", "content": f"You create engaging, age-appropriate decision points for interactive children's stories targeted at {age}-year-olds."},
                {"role": "user", "content": prompt}
            ],
            temperature=self.temperature,
            max_tokens=512,
            response_format={"type": "json_object"}
        )

        # Extract and parse the JSON response
        try:
            decision_point_text = response.choices[0].message.content
            # Use json.loads for safer parsing than eval
            import json
            decision_point = json.loads(decision_point_text)

            # Add metadata including moral/recovery info (Updated metadata)
            decision_point["metadata"] = {
                "segment_index": segment_index,
                "narrative_stage": narrative_stage,
                "decision_type": selected_type,
                "model_used": self.model_name,
                "is_moral_decision": is_moral_decision,
                "moral_choice_index": moral_choice_index if is_moral_decision else None,
                "needs_recovery": needs_recovery_flag, # Use the flag determined earlier
                "is_recovery_point": is_recovery_point
            }

            return decision_point

        except Exception as e:
            # Fallback response if parsing fails (Updated metadata)
            print(f"Error parsing decision point: {e}")
            # Determine fallback needs_recovery based on stage
            fallback_needs_recovery = stage_info.get("recovery_required", False) if stage_info.get("moral_guidance_required", False) else False
            return {
                "decision_prompt": f"What should {main_character} do next?",
                "options": [
                    {
                        "choice_text": "Take the safe path",
                        "consequence_summary": "This might be easier but less exciting."
                    },
                    {
                        "choice_text": "Try something risky",
                        "consequence_summary": "This could lead to adventure or trouble."
                    }
                ],
                "metadata": {
                    "segment_index": segment_index,
                    "narrative_stage": narrative_stage,
                    "decision_type": "action",
                    "model_used": self.model_name,
                    "is_moral_decision": False, # Fallback assumes not moral
                    "moral_choice_index": None,
                    "needs_recovery": fallback_needs_recovery, # Use fallback value
                    "is_recovery_point": is_recovery_point,
                    "error": str(e)
                }
            }

    # Merged method
    def generate_continuation_from_choice(
        self,
        branching_engine: 'BranchingLogicEngine', # Use BranchingLogicEngine instance
        decision_prompt: str,
        selected_choice: Dict[str, str],
        unselected_choice: Dict[str, str],
        character_names: List[str],
        age: int,
        next_segment_length: int = 4,  # Number of sentences
        is_recovery_path: bool = False # Added parameter
    ) -> str:
        """
        Generate a continuation of the story based on the user's selected choice.

        Args:
            branching_engine: Instance of the BranchingLogicEngine to get context
            decision_prompt: The question that was asked
            selected_choice: Dict with choice_text and consequence_summary that user selected
            unselected_choice: Dict with choice_text and consequence_summary that user did not select
            character_names: List of character names
            age: Target audience age
            next_segment_length: Approximate length of next segment in sentences
            is_recovery_path: Whether this is a continuation after a wrong moral choice

        Returns:
            String containing the next story segment
        """
        # Get potentially summarized context from the engine
        story_context_for_llm = branching_engine.get_story_context_for_llm()

        # Ensure appropriate segment length based on age (Unchanged from original)
        if age <= 5:
            actual_segment_length = min(next_segment_length, 3)
        elif age <= 8:
            actual_segment_length = min(next_segment_length, 4)
        else:
            actual_segment_length = next_segment_length

        # Add more guidance for younger children (Unchanged from original)
        if age <= 5:
            tone_guidance = "Use very simple vocabulary and short sentences. Focus on clear cause and effect."
        elif age <= 8:
            tone_guidance = "Use straightforward language with some descriptive elements. Include character emotions."
        else:
            tone_guidance = "Use more complex vocabulary and sentence structure. Include character thoughts and feelings."

        # Add special guidance for recovery paths (Added logic)
        recovery_guidance = ""
        if is_recovery_path:
             # Set the internal flag for the next decision point generation
             self.needs_recovery = True
             recovery_guidance = """
             IMPORTANT: This is a recovery segment after the reader made a choice that didn't align with the moral lesson.

             Show the consequences of the choice, but create a situation where the character realizes they should
             reconsider their approach. Don't be judgmental or heavy-handed, but subtly guide the story back
             toward a positive lesson.

             End the segment in a way that naturally leads to another decision point where the character can
             make a better choice aligned with the story's moral.
             """
        else:
            # Reset recovery flag if not on a recovery path
            self.needs_recovery = False


        # Use story_context_for_llm from engine instead of passed context
        prompt = f"""
        Continue this children's story for {age}-year-olds.

        STORY SO FAR:
        {story_context_for_llm}

        DECISION POINT:
        {decision_prompt}

        THE READER CHOSE:
        "{selected_choice['choice_text']}"
        Expected consequence: {selected_choice['consequence_summary']}

        {recovery_guidance}

        Write the next segment of the story (about {actual_segment_length} sentences) that:
        1. Directly shows the outcome of the chosen option
        2. Is age-appropriate for {age}-year-olds
        3. {tone_guidance}
        4. Advances the story in a meaningful way that follows from the choice
        5. Is engaging and visual

        Return only the story text with no additional explanation or commentary.
        """

        response = self.groq_client.chat.completions.create(
            model=self.model_name,
            messages=[
                {"role": "system", "content": f"You are an expert children's storyteller writing for {age}-year-olds."},
                {"role": "user", "content": prompt}
            ],
            temperature=self.temperature,
            max_tokens=400
        )

        next_segment = response.choices[0].message.content.strip()
        return next_segment


# Example usage
if __name__ == "__main__":
    # Initialize the decision point generator
    generator = DecisionPointGenerator(
        api_key=os.environ.get("GROQ_API_KEY", "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")
    )

    branching_engine_instance = BranchingLogicEngine(
         initial_segment="Luna is a young rabbit exploring the forest for the first time.", # Provide some initial data
         story_metadata={"theme": "curiosity"}, # Add some metadata
         api_key=os.environ.get("GROQ_API_KEY", "YOUR_FALLBACK_API_KEY") # Pass API key
    )

    # Example story segment and context
    sample_segment = "Luna the rabbit hopped excitedly through the forest. She found a shiny red apple under a tall oak tree. The apple looked delicious, but Luna remembered her mother told her not to eat strange food from the forest."

    sample_context = "Luna is a young rabbit exploring the forest for the first time. Her parents warned her about forest dangers, but she's curious and adventurous."

    decision = generator.generate_decision_points(
        story_segment=sample_segment,
        branching_engine=branching_engine_instance, # Pass the actual engine instance
        character_names=["Luna", "Mother Rabbit"],
        age=6,
        segment_index=1, # Example index
        total_segments=5, # Example total
        story_length="short" # Example length
    )

    if decision and "options" in decision: # Check if decision and options exist
        print("Decision Prompt:", decision.get("decision_prompt", "N/A"))
        print("\nOptions:")
        options = decision.get("options", [])
        if len(options) > 0:
            print("1.", options[0].get("choice_text", "N/A"))
            print("   ", options[0].get("consequence_summary", ""))
        if len(options) > 1:
            print("2.", options[1].get("choice_text", "N/A"))
            print("   ", options[1].get("consequence_summary", ""))

        # Example: Generate continuation from selected choice - Pass the engine instance
        if len(options) > 0:
            # Ensure the branching_engine_instance has the current segment added if needed for context
            # (In a real scenario, the engine state would be updated before this call)
            # For this standalone example, the context comes from the engine's initial state + get_full_story_text logic

            next_segment = generator.generate_continuation_from_choice(
                branching_engine=branching_engine_instance, # Pass the engine instance
                decision_prompt=decision.get("decision_prompt", "N/A"),
                selected_choice=options[0],
                unselected_choice=options[1] if len(options) > 1 else options[0],
                character_names=["Luna", "Mother Rabbit"],
                age=6
            )

            print("\nContinuation based on first choice:")
            print(next_segment)
        else:
            print("\nCould not generate continuation example: No options found.")
    else:
        print("\nError: Failed to generate decision point or options in example.")
        print("Received decision data:", decision)

Decision Prompt: What should Luna do with the shiny red apple?

Options:
1. Take a small bite to see if it's good
    Luna might learn it's a special forest treat, and her curiosity grows
2. Leave the apple behind because it's not from her garden
    Luna shows she trusts her mother's advice and is willing to make responsible choices

Continuation based on first choice:
Luna took a small nibble of the shiny red apple. Mmm... it was the most delicious thing she had ever tasted! The sweetness exploded in her mouth and the crunch of the apple was so satisfying. Luna's eyes grew wide with excitement and she couldn't help but take another bite, and another, until the apple was almost gone.


In [14]:
"""
Moral Generation System
----------------------
This module extracts and generates a moral based on the story and decisions.
It leverages the existing StoryRAG class to retrieve similar morals and uses
the Gemma model via Groq API for moral synthesis and reflection.

Dependencies:
- groq
- typing
- existing StoryRAG infrastructure
"""

import groq
from typing import Dict, List, Any, Optional, Tuple
import os
from pathlib import Path
import pandas as pd
import numpy as np
import re


class MoralGenerator:
    """
    Generates age-appropriate morals for interactive children's stories
    using the StoryRAG retrieval system and Gemma model via Groq.

    Attributes:
        groq_client: Initialized Groq client for API calls
        story_rag: StoryRAG instance for retrieval of similar morals
        morals_df: DataFrame containing processed morals (if direct access needed)
        model_name: The Groq model to use for generation (default: gemma2-9b-it)
        temperature: Temperature for generation (controls creativity)
        data_dir: Directory containing processed data files
    """

    def __init__(
        self,
        api_key: str = "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",
        story_rag = None,  # StoryRAG instance
        processed_data_dir: str = "data/processed",
        model_name: str = "gemma2-9b-it",
        temperature: float = 0.6
    ):
        """
        Initialize the MoralGenerator with Groq API access and configuration.

        Args:
            api_key: Groq API key
            story_rag: Instance of the StoryRAG class for retrieval
            processed_data_dir: Directory containing processed data files (default: "data/processed")
            model_name: Groq model to use (default: gemma2-9b-it)
            temperature: Controls randomness in generation (default: 0.6)
        """
        self.groq_client = groq.Client(api_key=api_key)
        self.story_rag = story_rag
        self.data_dir = Path(processed_data_dir)
        self.model_name = model_name
        self.temperature = temperature

        # Optionally load the morals dataframe directly for reference
        try:
            self.morals_df = pd.read_csv(self.data_dir / "processed_morals.csv")
            print(f"Successfully loaded morals dataset with {len(self.morals_df)} entries")
        except FileNotFoundError:
            print("Warning: processed_morals.csv not found. Will rely on StoryRAG for retrievals.")
            self.morals_df = None

    def extract_themes(self, story_text: str, choices_made: List[Dict[str, Any]]) -> List[str]:
        """
        Extract the central themes from the story and choices made.

        Args:
            story_text: The full text of the story
            choices_made: List of decision choices that were selected

        Returns:
            List of identified themes
        """
        # Prepare a summary of choices made to help with theme identification
        choices_summary = ""
        if choices_made:
            choices_summary = "The reader made these choices during the story:\n"
            for i, choice in enumerate(choices_made, 1):
                choices_summary += f"{i}. {choice.get('choice_text', '')}\n"

        prompt = f"""
        Analyze this children's story and identify the 3-5 central themes.

        STORY TEXT:
        {story_text}

        {choices_summary}

        Identify the main themes that are present in this story (like courage, friendship, honesty, etc.).
        Consider both the events in the story and the choices the reader made.

        Return your response as a comma-separated list of themes, for example:
        bravery, kindness, perseverance, trust

        THEMES:
        """

        response = self.groq_client.chat.completions.create(
            model=self.model_name,
            messages=[
                {"role": "system", "content": "You are an expert at analyzing children's stories and extracting their central themes."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.4,  # Lower temperature for consistent theme extraction
            max_tokens=100
        )

        # Process the response to extract themes
        themes_text = response.choices[0].message.content.strip()

        # Clean up the response and split into individual themes
        themes = [
            theme.strip().lower()
            for theme in re.split(r',|\n', themes_text)
            if theme.strip()
        ]

        return themes

    def retrieve_similar_morals(self, themes: List[str], top_k: int = 3) -> List[Dict[str, Any]]:
        """
        Use StoryRAG to retrieve morals similar to the identified themes.

        Args:
            themes: List of themes extracted from the story
            top_k: Number of similar morals to retrieve

        Returns:
            List of retrieved morals with metadata
        """
        # Check if StoryRAG is available
        if not self.story_rag:
            print("Warning: No StoryRAG instance provided, returning default morals")
            return [
                {"moral": "It's important to be brave and face your fears.", "theme": "courage"},
                {"moral": "True friendship means being there for others when they need help.", "theme": "friendship"},
                {"moral": "Kindness to others brings joy to everyone.", "theme": "kindness"}
            ]

        # Use the primary theme to retrieve relevant morals
        similar_morals = []

        # Try each theme until we get enough morals
        for theme in themes:
            # Use StoryRAG's get_moral_by_theme method to find matching morals
            theme_morals = self.story_rag.get_moral_by_theme(theme, top_k=top_k)

            if theme_morals:
                similar_morals.extend(theme_morals)

            # If we have enough morals, stop
            if len(similar_morals) >= top_k:
                break

        # If we still don't have enough morals, try a more general search
        if len(similar_morals) < top_k:
            # Use a more general theme like "life lessons"
            more_morals = self.story_rag.get_moral_by_theme("life lessons", top_k=top_k-len(similar_morals))
            if more_morals:
                similar_morals.extend(more_morals)

        # Ensure we don't return duplicates and limit to top_k
        unique_morals = []
        moral_texts = set()

        for moral in similar_morals:
            moral_text = moral.get("moral", "")
            if moral_text and moral_text not in moral_texts:
                moral_texts.add(moral_text)
                unique_morals.append(moral)
                if len(unique_morals) >= top_k:
                    break

        return unique_morals

    def generate_moral(
        self,
        story_text: str,
        choices_made: List[Dict[str, Any]],
        age: int,
        theme: str = None,
        story_title: str = None
    ) -> Dict[str, Any]:
        """
        Generate a single, clean age-appropriate moral for the story.

        Args:
            story_text: The full text of the story
            choices_made: List of decision choices that were selected
            age: Target audience age for appropriate moral complexity
            theme: Optional main theme to focus on
            story_title: Optional title of the story

        Returns:
            Dictionary containing the generated moral and metadata
        """
        # First, extract themes if not provided
        story_themes = [theme.lower()] if theme else []
        extracted_themes = self.extract_themes(story_text, choices_made)

        # Combine provided theme with extracted themes, prioritizing the provided theme
        if theme and theme.lower() not in extracted_themes:
            story_themes.extend(extracted_themes)
        else:
            story_themes = extracted_themes

        # Create a brief summary of the story for context
        summary_prompt = f"""
        Summarize this children's story in a few sentences:

        {story_text}

        Summary:
        """

        summary_response = self.groq_client.chat.completions.create(
            model=self.model_name,
            messages=[
                {"role": "system", "content": "You summarize children's stories concisely."},
                {"role": "user", "content": summary_prompt}
            ],
            temperature=0.4,
            max_tokens=150
        )

        story_summary = summary_response.choices[0].message.content.strip()

        # Retrieve similar morals using StoryRAG
        similar_morals = self.retrieve_similar_morals(story_themes)

        # Extract the text of the retrieved morals
        moral_examples = []
        for moral in similar_morals:
            if 'moral' in moral:
                moral_examples.append(moral['moral'])
            else:
                for key, value in moral.items():
                    if isinstance(value, str) and len(value) > 10:
                        moral_examples.append(value)
                        break

        # Format the moral examples for the prompt
        moral_examples_text = ""
        if moral_examples:
            moral_examples_text = "Here are some example morals from similar stories:\n"
            for i, example in enumerate(moral_examples[:3], 1):
                moral_examples_text += f"Example {i}: {example}\n\n"

        # Adjust complexity based on age
        if age <= 5:
            complexity = "very simple, using basic vocabulary for very young children (ages 3-5)"
        elif age <= 8:
            complexity = "straightforward but meaningful for early elementary children (ages 6-8)"
        else:
            complexity = "more nuanced with a deeper message for older elementary children (ages 9-12)"

        title_info = f"titled '{story_title}'" if story_title else ""

        # Modified prompt to request a single, clear moral
        prompt = f"""
        Create a single moral lesson for a children's story {title_info}.

        STORY SUMMARY:
        {story_summary}

        MAIN THEMES:
        {', '.join(story_themes)}

        {moral_examples_text}

        Provide ONE clear moral statement that is:
        1. {complexity}
        2. Directly connected to the story's themes and events
        3. Positive and encouraging
        4. Brief (1-2 sentences)
        5. Relatable to children's everyday experiences

        Do NOT provide multiple options or numbered alternatives.
        Do NOT use formatting like asterisks or bullet points.
        Respond with ONLY the moral statement itself.

        MORAL:
        """

        response = self.groq_client.chat.completions.create(
            model=self.model_name,
            messages=[
                {"role": "system", "content": f"You create meaningful, age-appropriate morals for children's stories targeted at {age}-year-olds."},
                {"role": "user", "content": prompt}
            ],
            temperature=self.temperature,
            max_tokens=200
        )

        moral_text = response.choices[0].message.content.strip()

        # Clean the moral to remove any remaining formatting
        moral_text = re.sub(r'Option \d+.*?:', '', moral_text)
        moral_text = re.sub(r'Here are.*?options!', '', moral_text)
        moral_text = moral_text.replace('*', '').strip()

        # Return the generated moral with metadata
        return {
            "moral": moral_text,
            "themes": story_themes,
            "similar_morals_used": moral_examples[:3] if moral_examples else [],
            "metadata": {
                "age": age,
                "model_used": self.model_name,
                "story_summary": story_summary
            }
        }

    def generate_expanded_moral_content(
        self,
        moral: str,
        themes: List[str],
        age: int,
        format_type: str = "explanation"
    ) -> str:
        """
        Generate expanded content based on the moral for different use cases.

        Args:
            moral: The main moral text
            themes: List of story themes
            age: Target audience age
            format_type: Type of expanded content ("explanation", "discussion_questions", "activity")

        Returns:
            Expanded content text
        """
        if format_type == "explanation":
            prompt_focus = f"""
            Create a child-friendly explanation of this moral suitable for {age}-year-olds.
            Explain what the moral means in simple terms and provide a real-life example that children can relate to.
            Keep it concise (3-5 sentences total).
            """

        elif format_type == "discussion_questions":
            prompt_focus = f"""
            Create 3 discussion questions about this moral suitable for {age}-year-olds.
            The questions should help children reflect on the moral and connect it to their own experiences.
            Make the questions open-ended and thought-provoking but simple enough for the age group.
            """

        elif format_type == "activity":
            prompt_focus = f"""
            Create a simple activity related to this moral suitable for {age}-year-olds.
            The activity should help reinforce the moral through creative expression or roleplay.
            Describe the activity in 3-5 sentences, including any simple materials needed.
            """

        else:
            prompt_focus = f"""
            Provide additional insight into this moral suitable for {age}-year-olds.
            Explain the importance of the moral in children's development and growth.
            Keep it concise (3-5 sentences).
            """

        prompt = f"""
        MORAL:
        {moral}

        THEMES:
        {', '.join(themes)}

        TASK:
        {prompt_focus}

        Ensure your response is:
        1. Age-appropriate for {age}-year-olds
        2. Positive and encouraging
        3. Clear and simple to understand
        """

        response = self.groq_client.chat.completions.create(
            model=self.model_name,
            messages=[
                {"role": "system", "content": f"You create educational content for children aged {age}."},
                {"role": "user", "content": prompt}
            ],
            temperature=self.temperature,
            max_tokens=300
        )

        return response.choices[0].message.content.strip()


# Example usage
if __name__ == "__main__":
    # This is a simplified version of the StoryRAG class for testing
    class SimpleStoryRAG:
        def get_moral_by_theme(self, theme, top_k=3):
            # Simple mocked implementation
            morals_by_theme = {
                "friendship": [
                    {"moral": "True friendship means being there for others when they need help.", "theme": "friendship"},
                    {"moral": "Friends accept each other's differences.", "theme": "friendship"},
                    {"moral": "Friendship can grow in unlikely places.", "theme": "friendship"}
                ],
                "courage": [
                    {"moral": "It's important to be brave and face your fears.", "theme": "courage"},
                    {"moral": "Courage means doing what's right even when it's hard.", "theme": "courage"},
                    {"moral": "Being brave doesn't mean you're never scared.", "theme": "courage"}
                ],
                "kindness": [
                    {"moral": "Kindness to others brings joy to everyone.", "theme": "kindness"},
                    {"moral": "Small acts of kindness can make a big difference.", "theme": "kindness"},
                    {"moral": "Being kind is more important than being right.", "theme": "kindness"}
                ]
            }

            # Return morals for the requested theme or empty list if not found
            return morals_by_theme.get(theme.lower(), [])[:top_k]

    # Create a mock story
    sample_story = """
    Once upon a time, there was a rabbit named Luna who lived at the edge of the Whispering Forest.
    One morning, Luna decided to explore deeper into the forest even though she was a little scared.
    As she ventured in, she found a beautiful clearing with a stream. There, she met a lost fox cub who was crying.
    Even though rabbits and foxes weren't usually friends, Luna decided to help the fox cub find its way home.
    Together, they faced challenges and helped each other overcome obstacles.
    When they finally found the fox family, they were so grateful to Luna that they promised never to hunt rabbits again.
    Luna and the fox cub became unlikely friends, showing everyone in the forest that kindness and courage can bring different animals together.
    """

    sample_choices = [
        {"choice_text": "Explore deeper into the forest"},
        {"choice_text": "Help the fox cub"},
        {"choice_text": "Cross the dangerous stream"}
    ]

    # Initialize the moral generator with the mock StoryRAG
    moral_gen = MoralGenerator(
        api_key="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",
        story_rag=SimpleStoryRAG(),
        processed_data_dir="data/processed"
    )

    # Generate a moral for the sample story
    result = moral_gen.generate_moral(
        story_text=sample_story,
        choices_made=sample_choices,
        age=7,
        story_title="Luna and the Lost Fox Cub"
    )

    # Print the results
    print(f"Themes identified: {', '.join(result['themes'])}")
    print(f"\nGenerated moral: {result['moral']}")

    print("\nSimilar morals that were referenced:")
    for i, moral in enumerate(result['similar_morals_used'], 1):
        print(f"{i}. {moral}")

    # Generate expanded content
    explanation = moral_gen.generate_expanded_moral_content(
        moral=result['moral'],
        themes=result['themes'],
        age=7,
        format_type="explanation"
    )

    discussion_questions = moral_gen.generate_expanded_moral_content(
        moral=result['moral'],
        themes=result['themes'],
        age=7,
        format_type="discussion_questions"
    )

    print("\nMoral explanation for children:")
    print(explanation)

    print("\nDiscussion questions:")
    print(discussion_questions)

Successfully loaded morals dataset with 665 entries
Themes identified: here are the main themes present in the story:, **courage, kindness, friendship, overcoming prejudice, helping others**, let me explain how these themes are reflected in the story:, * **courage:** luna chooses to explore the unknown forest despite being scared, and she helps the fox cub even though foxes are predators., * **kindness:** luna's actions towards the lost fox cub demonstrate her compassion and willingness to help despite their differences., * **friendship:** luna and the

Generated moral: Helping someone in need can build a strong friendship, even if they are different from you.

Similar morals that were referenced:
1. Kindness to others brings joy to everyone.
2. Small acts of kindness can make a big difference.
3. Being kind is more important than being right.

Moral explanation for children:
Being kind to someone who needs help can make a great friend, even if they are different from you! It takes cou

In [16]:
!pip install gTTS

In [17]:
import os
import random
import time
import base64
import tempfile
from typing import List, Dict, Any, Optional
import gradio as gr
from gtts import gTTS
import io


GROQ_API_KEY = os.environ.get("GROQ_API_KEY")

# Check if required classes exist in the notebook scope
def check_required_classes():
    """Check if the required classes exist in the current notebook scope"""
    missing_classes = []
    required_classes = [
        ("StoryGenerator", "story generation module"),
        ("DecisionPointGenerator", "decision point module"),
        ("BranchingLogicEngine", "branching logic engine"),
        ("MoralGenerator", "moral generation system"),
        ("StoryRAG", "story retrieval augmented generation")
    ]

    for class_name, module_desc in required_classes:
        if class_name not in globals():
            missing_classes.append(f"'{class_name}' from {module_desc}")

    if missing_classes:
        error_message = "The following classes are missing from the notebook environment:\n"
        error_message += "\n".join(missing_classes)
        error_message += "\n\nPlease make sure you've run all cells that define these classes."
        raise NameError(error_message)

    return "✅ All required classes are available in the notebook environment."


class InteractiveStorytellingSystem:
    """
    Main class to integrate all components of the interactive storytelling system.

    This integrates StoryGenerator, DecisionPointGenerator, BranchingLogicEngine,
    and MoralGenerator using a real StoryRAG instance for retrieval.
    """

    def __init__(self, api_key: str = GROQ_API_KEY, data_dir: str = "data/processed"):
        """
        Initialize the complete storytelling system.

        Args:
            api_key: Groq API key for LLM access
            data_dir: Directory containing processed data files
        """
        self.api_key = api_key
        self.data_dir = data_dir

        # Initialize the StoryRAG component for retrieval
        print("Initializing StoryRAG...")
        self.story_rag = StoryRAG(data_dir=data_dir)

        # Initialize the StoryGenerator
        print("Initializing StoryGenerator...")
        self.story_generator = StoryGenerator(
            api_key=api_key,
            data_dir=data_dir
        )

        # Initialize the DecisionPointGenerator
        print("Initializing DecisionPointGenerator...")
        self.decision_generator = DecisionPointGenerator(
            api_key=api_key
        )

        # Initialize the MoralGenerator with the StoryRAG
        print("Initializing MoralGenerator...")
        self.moral_generator = MoralGenerator(
            api_key=api_key,
            story_rag=self.story_rag,
            processed_data_dir=data_dir
        )

        # This will be initialized when generating a story
        self.branching_engine = None

    def generate_story(
        self,
        age: int,
        theme: str,
        length: str = "medium",
        characters: Optional[List[str]] = None
    ) -> Dict[str, Any]:
        """
        Generate the initial story structure.

        Args:
            age: Target audience age
            theme: Main story theme
            length: "short", "medium", or "long"
            characters: Optional list of character names

        Returns:
            Generated story data
        """
        print(f"Generating {length} story about '{theme}' for age {age}...")

        # Generate the initial story
        story_data = self.story_generator.generate_story(
            age=age,
            theme=theme,
            length=length,
            characters=characters
        )

        return story_data

    def initialize_story_engine(self, story_data: Dict[str, Any]) -> None:
        # Initialize the branching engine, passing the API key
        self.branching_engine = BranchingLogicEngine(
            initial_segment=story_data["segments"][0],
            story_metadata={
                "title": story_data["title"],
                "age": story_data["metadata"]["age"],
                "theme": story_data["metadata"]["theme"],
                "length": story_data["metadata"]["length"],
                "characters": story_data["metadata"]["characters"],
                "api_key": self.api_key # Pass API key
            }
        )

        # Add remaining initial segments to the engine
        current_node_id = self.branching_engine.root_node_id

        for segment in story_data["segments"][1:]:
            # Add each segment as a continuation without choices
            new_node_id = self.branching_engine.add_story_continuation(
                choice_index=0,  # Default path
                continuation_text=segment,
                log_choice=False
            )
            current_node_id = new_node_id

    def generate_decision_point(self, is_recovery_point: bool = False) -> Dict[str, Any]:
        """
        Generate a decision point for the current story node.

        Returns:
            Decision point data
        """
        # Get the current story state
        current_node = self.branching_engine.get_current_node()
        story_context = self.branching_engine.get_full_story_text()
        current_segment = current_node.segment

        # Get metadata
        metadata = self.branching_engine.story_metadata
        story_length = metadata.get("length", "medium")
        age = metadata.get("age", 8)
        characters = metadata.get("characters", [])

        # Get position in story
        path = self.branching_engine.path_history
        segment_index = len(path) - 1
        total_segments = max(10, int(len(path) * 1.5))  # Estimate total segments

        # Generate decision point
        decision_point = self.decision_generator.generate_decision_points(
            story_segment=current_segment,
            # story_context=story_context, # Remove this
            branching_engine=self.branching_engine, # Pass the engine
            character_names=characters,
            age=age,
            segment_index=segment_index,
            total_segments=total_segments,
            story_length=story_length,
            is_recovery_point=is_recovery_point
        )

        # Add decision to current node
        self.branching_engine.add_decision_to_current_node(decision_point)

        return decision_point

    def select_choice(self, choice_index: int) -> str:
        """
        Process the user's choice and generate the next story segment.

        Args:
            choice_index: Index of the selected choice (0 or 1)

        Returns:
            The text of the next story segment
        """
        # Get current state
        current_node = self.branching_engine.get_current_node()
        decision = current_node.decisions
        # story_so_far = self.branching_engine.get_full_story_text() # Context now handled by engine

        # Get metadata
        metadata = self.branching_engine.story_metadata
        age = metadata.get("age", 8)
        characters = metadata.get("characters", [])

        # Selected and unselected choices
        selected_choice = decision["options"][choice_index]
        unselected_choice = decision["options"][1 - choice_index]  # The other choice

        # --- START FIX ---
        # Determine if the selected path is a recovery path
        # Check the metadata stored within the decision itself
        decision_metadata = decision.get("metadata", {})
        needs_recovery = decision_metadata.get("needs_recovery", False)
        moral_choice_index = decision_metadata.get("moral_choice_index", None)

        # The path is a recovery path IF it was marked as needing recovery
        # AND the user chose the *incorrect* moral option.
        is_recovery_path = False
        if needs_recovery and moral_choice_index is not None:
             if choice_index != moral_choice_index:
                 is_recovery_path = True
        # --- END FIX ---


        # Generate continuation based on choice
        next_segment = self.decision_generator.generate_continuation_from_choice(
            # story_so_far=story_so_far, # Remove this
            branching_engine=self.branching_engine, # Pass the engine
            decision_prompt=decision["decision_prompt"],
            selected_choice=selected_choice,
            unselected_choice=unselected_choice,
            character_names=characters,
            age=age,
            is_recovery_path=is_recovery_path # Pass the argument here
        )

        # Add continuation to the branching engine
        self.branching_engine.add_story_continuation(
            choice_index=choice_index,
            continuation_text=next_segment,
            metadata={"selected_choice": selected_choice}
        )

        return next_segment

    def generate_story_moral(self) -> Dict[str, Any]:
        """
        Generate a moral for the complete story.

        Returns:
            Dictionary with the moral and related information
        """
        # Get the full story text
        story_text = self.branching_engine.get_full_story_text()

        # Get metadata
        metadata = self.branching_engine.story_metadata
        age = metadata.get("age", 8)
        theme = metadata.get("theme", "")
        title = metadata.get("title", "")

        # Get choices made
        choices_made = []
        for choice in self.branching_engine.choice_history:
            node_id = choice["from_node"]
            choice_index = choice["choice_index"]

            if node_id in self.branching_engine.nodes:
                node = self.branching_engine.nodes[node_id]
                if node.decisions and "options" in node.decisions:
                    choices_made.append(node.decisions["options"][choice_index])

        # Generate moral
        moral_data = self.moral_generator.generate_moral(
            story_text=story_text,
            choices_made=choices_made,
            age=age,
            theme=theme,
            story_title=title
        )

        # Generate expanded content
        explanation = self.moral_generator.generate_expanded_moral_content(
            moral=moral_data["moral"],
            themes=moral_data["themes"],
            age=age,
            format_type="explanation"
        )

        discussion = self.moral_generator.generate_expanded_moral_content(
            moral=moral_data["moral"],
            themes=moral_data["themes"],
            age=age,
            format_type="discussion_questions"
        )

        # Add to the result
        moral_data["explanation"] = explanation
        moral_data["discussion_questions"] = discussion

        return moral_data


# Text-to-Speech helper functions
def generate_speech(text, lang='en', slow=False):
    """
    Generate speech audio from text using gTTS

    Args:
        text: Text to convert to speech
        lang: Language code (default: 'en')
        slow: Whether to read the text slowly (default: False)

    Returns:
        Audio data as bytes
    """
    try:
        tts = gTTS(text=text, lang=lang, slow=slow)
        audio_data = io.BytesIO()
        tts.write_to_fp(audio_data)
        audio_data.seek(0)
        return audio_data.read()
    except Exception as e:
        print(f"Error generating speech: {str(e)}")
        return None


def get_audio_html(audio_data, segment_id):
    """
    Convert audio data to HTML audio element

    Args:
        audio_data: Audio data as bytes
        segment_id: Unique identifier for the audio element

    Returns:
        HTML audio element as string
    """
    if audio_data is None:
        return ""

    # Convert audio data to base64
    audio_b64 = base64.b64encode(audio_data).decode("utf-8")

    # Create HTML audio element
    audio_html = f"""
    <div class="audio-player" style="margin: 5px 0;">
        <audio id="audio-{segment_id}" controls>
            <source src="data:audio/mp3;base64,{audio_b64}" type="audio/mp3">
            Your browser does not support the audio element.
        </audio>
    </div>
    """

    return audio_html


# Helper functions for formatting HTML
def format_title(title):
    """Format story title as HTML"""
    # Generate speech for title
    audio_data = generate_speech(title)
    audio_html = get_audio_html(audio_data, "title")

    return f"""
    <h1 style='color: #2E86C1; text-align: center'>{title}</h1>
    <div style='text-align: center;'>{audio_html}</div>
    """


def format_segment(segment, index=None):
    """Format story segment as HTML with audio player"""
    # Generate unique ID for this segment
    segment_id = f"segment-{index}" if index is not None else f"segment-{random.randint(1000, 9999)}"

    # Generate speech for segment
    audio_data = generate_speech(segment)
    audio_html = get_audio_html(audio_data, segment_id)

    if index is not None:
        return f"""
        <div style='background-color: #EBF5FB; border-left: 4px solid #3498DB; padding: 10px; margin: 10px 0;'>
            <h3>Part {index}</h3>
            <p>{segment}</p>
            {audio_html}
        </div>
        """
    else:
        return f"""
        <div style='background-color: #EBF5FB; border-left: 4px solid #3498DB; padding: 10px; margin: 10px 0;'>
            <p>{segment}</p>
            {audio_html}
        </div>
        """


def format_decision(decision_prompt, options):
    """Format decision prompt as HTML with audio"""
    # Generate speech for decision prompt
    audio_data = generate_speech(decision_prompt)
    audio_html = get_audio_html(audio_data, f"decision-{random.randint(1000, 9999)}")

    html = f"""
    <div style='background-color: #FEF9E7; border-left: 4px solid #F1C40F; padding: 10px; margin: 10px 0;'>
        <h3>Decision Time!</h3>
        <p>{decision_prompt}</p>
        {audio_html}
    </div>
    """

    for i, option in enumerate(options):
        option_text = f"Option {i+1}: {option['choice_text']}. {option['consequence_summary']}"
        option_audio = get_audio_html(generate_speech(option_text), f"option-{i+1}-{random.randint(1000, 9999)}")

        html += f"""
        <div style='margin: 10px 0;'>
            <b>Option {i+1}:</b> {option['choice_text']}<br>
            <i>{option['consequence_summary']}</i>
            {option_audio}
        </div>
        """

    return html

def format_moral(moral_data):
    """Format moral and related content as HTML with audio"""
    # Get the clean, single moral text
    moral_text = moral_data['moral']

    # Generate speech for moral and supporting content
    moral_audio = get_audio_html(generate_speech(moral_text), "moral")
    explanation_audio = get_audio_html(generate_speech(moral_data['explanation']), "explanation")

    # Format the discussion questions with line breaks
    formatted_discussion = moral_data['discussion_questions']
    formatted_discussion = formatted_discussion.replace('*', '')
    formatted_discussion = formatted_discussion.replace("1. ", "<br>1. ")
    formatted_discussion = formatted_discussion.replace("2. ", "<br>2. ")
    formatted_discussion = formatted_discussion.replace("3. ", "<br>3. ")
    if formatted_discussion.startswith("<br>"):
        formatted_discussion = formatted_discussion[4:]

    discussion_audio = get_audio_html(generate_speech(moral_data['discussion_questions']), "discussion")

    html = f"""
    <div style='background-color: #EAFAF1; border-left: 4px solid #27AE60; padding: 15px; margin: 15px 0;'>
        <h2 style='color: #27AE60;'>The Moral of the Story</h2>
        <p style='font-size: 1.2em; font-style: italic;'>{moral_text}</p>
        {moral_audio}
    </div>

    <div style='background-color: #F4F6F6; padding: 10px; margin: 10px 0;'>
        <h3>Explanation for Children</h3>
        <p>{moral_data['explanation']}</p>
        {explanation_audio}
    </div>

    <div style='background-color: #F4F6F6; padding: 10px; margin: 10px 0;'>
        <h3>Discussion Questions</h3>
        <p>{formatted_discussion}</p>
        {discussion_audio}
    </div>
    """
    return html


# Main Gradio interface for the interactive storytelling system
def create_gradio_interface():
    # Check if required classes exist
    try:
        check_message = check_required_classes()
    except NameError as e:
        check_message = str(e)

    # Initialize story state dictionary
    state = {
        'system': None,
        'story_data': None,
        'story_html': '',
        'current_segment_index': 0,
        'segments_shown': 0,
        'decision_shown': False,
        'story_finished': False,
        'error_message': check_message if 'missing' in check_message else None
    }

    # Generate story function
    def generate_story(age, theme, length, characters):
        # Check for errors
        if state['error_message']:
            return state['error_message'], gr.update(visible=False), gr.update(visible=False), gr.update(visible=False)

        try:
            # Reset state
            state['story_html'] = ''
            state['current_segment_index'] = 0
            state['segments_shown'] = 0
            state['decision_shown'] = False
            state['story_finished'] = False

            # Process characters
            character_list = [char.strip() for char in characters.split(',') if char.strip()]

            # Create system (no more mock objects)
            system = InteractiveStorytellingSystem(api_key=GROQ_API_KEY)
            state['system'] = system

            # Generate story
            story_data = system.generate_story(
                age=age,
                theme=theme,
                length=length,
                characters=character_list
            )
            state['story_data'] = story_data

            # Initialize story engine
            system.initialize_story_engine(story_data)

            # Start with title and first segment
            state['story_html'] = format_title(story_data['title'])
            state['story_html'] += format_segment(story_data['segments'][0], 1)
            state['segments_shown'] = 1
            state['current_segment_index'] = 1

            # Show continue button
            return state['story_html'], gr.update(visible=True), gr.update(visible=False), gr.update(visible=False)

        except Exception as e:
            return f"Error generating story: {str(e)}", gr.update(visible=False), gr.update(visible=False), gr.update(visible=False)

    # Continue story function
    def continue_story():
        system = state['system']
        story_data = state['story_data']

        try:
            # If we're still showing initial segments
            if state['segments_shown'] < len(story_data['segments']):
                # Show next segment
                next_index = state['segments_shown']
                state['story_html'] += format_segment(story_data['segments'][next_index], next_index + 1)
                state['segments_shown'] += 1
                state['current_segment_index'] += 1

                # If we've shown all segments, create first decision point
                if state['segments_shown'] >= len(story_data['segments']) and not state['decision_shown']:
                    # Generate first decision point
                    decision = system.generate_decision_point()
                    state['story_html'] += format_decision(decision['decision_prompt'], decision['options'])
                    state['decision_shown'] = True

                    # Show choice buttons
                    return state['story_html'], gr.update(visible=False), gr.update(visible=True), gr.update(visible=False)
                else:
                    # Still showing segments
                    return state['story_html'], gr.update(visible=True), gr.update(visible=False), gr.update(visible=False)
            else:
                # Check if we need to show another decision point
                if not state['decision_shown'] and not state['story_finished']:
                    # Generate decision point
                    decision = system.generate_decision_point()
                    state['story_html'] += format_decision(decision['decision_prompt'], decision['options'])
                    state['decision_shown'] = True

                    # Show choice buttons
                    return state['story_html'], gr.update(visible=False), gr.update(visible=True), gr.update(visible=False)
                else:
                    # Continue button was clicked but nothing to do
                    return state['story_html'], gr.update(visible=True), gr.update(visible=False), gr.update(visible=False)

        except Exception as e:
            return f"{state['story_html']}<div style='color: red;'>Error continuing story: {str(e)}</div>", gr.update(visible=True), gr.update(visible=False), gr.update(visible=False)

    # Make choice function
    def make_choice(choice_num):
        system = state['system']
        choice_index = choice_num - 1  # Convert to 0-indexed
        metadata = system.branching_engine.story_metadata
        story_length = metadata.get("length", "medium")

        # Get minimum required decisions based on story length
        min_decisions = system.decision_generator.get_decision_point_count(story_length)
        print("min dec", min_decisions)

        try:
            # Generate continuation based on choice
            next_segment = system.select_choice(choice_index)

            # Display the continuation
            state['current_segment_index'] += 1
            state['story_html'] += format_segment(next_segment, state['current_segment_index'])

            # Reset decision state
            state['decision_shown'] = False

            # Determine if story should end
            decision_count = len(system.branching_engine.choice_history)
            print(decision_count)

            # --- MODIFIED ENDING LOGIC ---
            should_end = False
            if decision_count < min_decisions:
                # Cannot end yet, minimum decisions not met
                should_end = False
            else:
                # Minimum decisions met, apply other ending conditions
                # End if not in recovery and segment count is high OR randomly based on length
                should_end = (
                    not system.decision_generator.needs_recovery and # <--- CORRECTED LINE
                    (state['current_segment_index'] >= 10) or # Adjusted threshold example
                     (state['current_segment_index'] > min_decisions*2) # Adjusted randomness
                )
            # --- END OF MODIFIED LOGIC ---


            if should_end:
                # End the story
                state['story_finished'] = True
                state['story_html'] += "<h3>The End</h3>"

                # Show finish button
                return state['story_html'], gr.update(visible=False), gr.update(visible=False), gr.update(visible=True)
            else:
                # Continue the story (needs another decision point)
                # Show the continue button to trigger the next decision point generation
                return state['story_html'], gr.update(visible=True), gr.update(visible=False), gr.update(visible=False)

        except Exception as e:
            return f"{state['story_html']}<div style='color: red;'>Error processing choice: {str(e)}</div>", gr.update(visible=True), gr.update(visible=False), gr.update(visible=False)

    # Finish story function
    def finish_story():
        system = state['system']

        try:
            # Generate and display moral
            moral_data = system.generate_story_moral()
            state['story_html'] += format_moral(moral_data)

            # Show create new story button (create a refresh button)
            return state['story_html'], gr.update(visible=False), gr.update(visible=False), gr.update(visible=False)

        except Exception as e:
            return f"{state['story_html']}<div style='color: red;'>Error generating moral: {str(e)}</div>", gr.update(visible=False), gr.update(visible=False), gr.update(visible=False)

    # Create Gradio interface
    with gr.Blocks(title="Interactive Storytelling System with Text-to-Speech") as demo:
        gr.Markdown("# Interactive Storytelling System with Text-to-Speech")
        gr.Markdown("This system generates interactive stories with choices, morals, and audio narration for children.")

        # Check for required classes
        if 'error_message' in state and state['error_message']:
            gr.Markdown(f"⚠ {state['error_message']}", elem_id="error-message")

        # Story generation form
        with gr.Row():
            with gr.Column(scale=1):
                age = gr.Slider(minimum=4, maximum=12, value=8, step=1, label="Age")
                length = gr.Dropdown(choices=["short", "medium", "long"], value="medium", label="Story Length")

            with gr.Column(scale=2):
                theme = gr.Textbox(label="Theme", placeholder="e.g., friendship and adventure", value="friendship and adventure")
                characters = gr.Textbox(label="Characters (comma separated)", placeholder="e.g., Luna the rabbit, Max the squirrel", value="Luna the rabbit, Max the squirrel")

        generate_btn = gr.Button("Generate Story", variant="primary")

        # Story display and interaction
        story_display = gr.HTML(label="Story")
        continue_btn = gr.Button("Continue", visible=False)

        with gr.Row(visible=False) as choice_row:
            choice1_btn = gr.Button("Choose Option 1", variant="secondary")
            choice2_btn = gr.Button("Choose Option 2", variant="secondary")

        finish_btn = gr.Button("See the Moral of the Story", visible=False, variant="primary")

        # Add a restart button for creating a new story after finishing
        restart_btn = gr.Button("Create New Story", visible=False, variant="primary")

        # Connect components
        generate_btn.click(
            generate_story,
            inputs=[age, theme, length, characters],
            outputs=[story_display, continue_btn, choice_row, finish_btn]
        )

        continue_btn.click(
            continue_story,
            outputs=[story_display, continue_btn, choice_row, finish_btn]
        )

        choice1_btn.click(
            lambda: make_choice(1),
            outputs=[story_display, continue_btn, choice_row, finish_btn]
        )

        choice2_btn.click(
            lambda: make_choice(2),
            outputs=[story_display, continue_btn, choice_row, finish_btn]
        )

        finish_btn.click(
            finish_story,
            outputs=[story_display, continue_btn, choice_row, finish_btn]
        )

        # Restart button reloads the page to start fresh
        restart_btn.click(
            fn=lambda: None,
            inputs=None,
            outputs=None,
            js="window.location.reload()"
        )

        # After finish_story is called, show the restart button
        finish_btn.click(
            lambda: gr.update(visible=True),
            None,
            restart_btn
        )

    return demo


# Run this in Google Colab
if __name__ == "__main__":
    try:
        # Make sure gTTS is installed
        try:
            import gtts
        except ImportError:
            print("Installing gTTS...")
            import pip
            pip.main(['install', 'gtts'])
            import gtts

        # Launch the interface
        demo = create_gradio_interface()
        demo.launch(debug=True, share=True)
    except Exception as e:
        print(f"Error setting up Gradio interface: {str(e)}")

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://542330c3cefd4898a0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Initializing StoryRAG...
Initializing StoryRAG with FAISS...
Loaded 4095 story sections
Loaded FAISS index with 4095 story vectors
Loaded 665 morals
Loaded FAISS index with 665 moral vectors
Loaded vocabulary for ages [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25]
Loaded word-to-age mapping with 31104 words
StoryRAG initialization complete!
Initializing StoryGenerator...
Initializing DecisionPointGenerator...
Initializing MoralGenerator...
Successfully loaded morals dataset with 665 entries
Generating long story about 'patience and trust' for age 12...
Context length (1322 words) exceeds threshold (400). Summarizing...
min dec 5
1
Context length (1456 words) exceeds threshold (400). Summarizing...
min dec 5
2
Context length (1546 words) exceeds threshold (400). Summarizing...
min dec 5
3
Context length (1635 words) exceeds threshold (400). Summarizing...
min dec 5
4
Context length (1721 words) exceeds threshold (400). Summarizing...
min dec 